In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
import numpy as np
from torchvision import datasets, transforms

import timm
from tqdm import tqdm
import os
import json
import datetime
import matplotlib.pyplot as plt
from typing import List, Optional

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
if device == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))


c:\Users\jkoui\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
GPU: NVIDIA GeForce RTX 4060 Ti


In [2]:
import random
GLOBAL_SEED = 2

def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # 让 cudnn 更可复现（会稍微慢一点）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_global_seed(GLOBAL_SEED)

In [3]:
try:
    GLOBAL_SEED
except NameError:
    GLOBAL_SEED = 42

def get_dataset(dataset_name, data_dir="./data", img_size=224):
    name = dataset_name.lower()

    if name == "imagenet":
        transform_train = transforms.Compose([
            transforms.RandomResizedCrop(img_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
        transform_val = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
        train_set = datasets.ImageFolder(f"{data_dir}/train", transform=transform_train)
        val_set = datasets.ImageFolder(f"{data_dir}/val", transform=transform_val)
        num_classes = len(train_set.classes)

    elif name == "cifar10":
        transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=(0.5, 0.5, 0.5),
                std=(0.5, 0.5, 0.5),
            )
        ])
        train_set = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
        val_set = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)
        num_classes = 10

    elif name == "cifar100":
        transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=(0.5, 0.5, 0.5),
                std=(0.5, 0.5, 0.5),
            )
        ])
        train_set = datasets.CIFAR100(root=data_dir, train=True, download=True, transform=transform)
        val_set = datasets.CIFAR100(root=data_dir, train=False, download=True, transform=transform)
        num_classes = 100

    elif name == "svhn":
        transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=(0.5, 0.5, 0.5),
                std=(0.5, 0.5, 0.5),
            )
        ])
        train_set = datasets.SVHN(root=data_dir, split='train', download=True, transform=transform)
        val_set   = datasets.SVHN(root=data_dir, split='test',  download=True, transform=transform)
        num_classes = 10

    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")

    return train_set, val_set, num_classes





def get_loaders(dataset_name, data_dir="./data", batch_size=64, img_size=224, num_workers=8):
    train_set, val_set, num_classes = get_dataset(dataset_name, data_dir, img_size)

    train_loader = DataLoader(
        train_set,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=(device == "cuda"),
    )
    val_loader = DataLoader(
        val_set,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=(device == "cuda"),
    )

    return train_loader, val_loader, num_classes


In [4]:
from tqdm import tqdm
import torch

def train_one_epoch(model, loader, criterion, optimizer, device, epoch, epochs, scaler=None):
    model.train()
    total_loss = 0.0
    correct = 0

    use_amp = scaler is not None

    loop = tqdm(loader, desc=f"Train [{epoch+1}/{epochs}]", ncols=100)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        if use_amp:
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(loader)
    acc = correct / len(loader.dataset)
    return avg_loss, acc


def eval_one_epoch(model, loader, criterion, device, epoch, epochs):
    model.eval()
    total_loss = 0.0
    correct = 0

    loop = tqdm(loader, desc=f"Val   [{epoch+1}/{epochs}]", ncols=100)
    with torch.no_grad():
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

            loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(loader)
    acc = correct / len(loader.dataset)
    return avg_loss, acc


In [5]:
# ============================================================
#  1. 谱分析基础工具：svd + effective rank + 画图
# ============================================================

def to_numpy(x):
    if isinstance(x, torch.Tensor):
        return x.detach().cpu().numpy()
    return np.array(x)


def compute_singular_values_from_qk(
    q: torch.Tensor,
    k: torch.Tensor,
    scale: float = 1.0,
    top_k: int = 32,
    batch_agg: str = "mean",
) -> torch.Tensor:
    """
    给定某一层的 Q, K，计算每个 head 的 pre-softmax S = (Q * scale) @ K^T 的前 top_k 个奇异值。

    参数：
      q, k: 形状 (B, H, N, D_h)
      scale: 一般 = 1/sqrt(D_h)
      top_k: 只保留前 top_k 个最大奇异值
      batch_agg:
        - "mean": S = mean_b (Q_b K_b^T)
        - "first": 只用第一个 batch

    返回：
      svals: (H, top_k)
    """
    assert q.shape == k.shape, "q, k 必须同形状 (B, H, N, D_h)"
    B, H, N, Dh = q.shape
    top_k = min(top_k, N)

    s_list = []

    for h in range(H):
        if batch_agg == "first":
            q_h = q[0, h] * scale      # (N, D_h)
            k_h = k[0, h]              # (N, D_h)
            S = q_h @ k_h.transpose(0, 1)  # (N, N)
        else:
            S = None
            for b in range(B):
                q_bh = q[b, h] * scale
                k_bh = k[b, h]
                S_b = q_bh @ k_bh.transpose(0, 1)
                if S is None:
                    S = S_b
                else:
                    S = S + S_b
            S = S / float(B)

        svals_full = torch.linalg.svdvals(S)   # (N,)
        svals_head = svals_full[:top_k]        # (top_k,)
        s_list.append(svals_head)

    svals = torch.stack(s_list, dim=0)  # (H, top_k)
    return svals


def normalize_singular_values(
    svals: torch.Tensor,
    mode: str = "sum",
    eps: float = 1e-12,
) -> torch.Tensor:
    """
    对奇异值做归一化，方便热力图看“谱形状”。

    mode:
      - "sum": 除以 sum(s_i) → 类似概率分布
      - "max": 除以 max(s_i) → 最大值变 1
      - "none": 不归一化
    """
    if mode == "none":
        return svals

    if mode == "sum":
        denom = svals.sum(dim=-1, keepdim=True) + eps
    elif mode == "max":
        denom = svals.max(dim=-1, keepdim=True).values + eps
    else:
        raise ValueError(f"Unknown normalize mode: {mode}")

    return svals / denom


def compute_effective_rank(
    svals: torch.Tensor,
    eps: float = 1e-12,
) -> torch.Tensor:
    """
    根据奇异值计算 effective rank：

      p_i = s_i / sum_j s_j
      H = - sum_i p_i log p_i
      erank = exp(H)

    参数：
      svals: (..., K)
    返回：
      erank: (...,)
    """
    svals = torch.clamp(svals, min=eps)
    S = svals.sum(dim=-1, keepdim=True) + eps
    p = svals / S
    log_p = torch.log(p + eps)
    H = -(p * log_p).sum(dim=-1)
    erank = torch.exp(H)
    return erank


def stack_svals_over_epochs(svals_per_epoch):
    """
    svals_per_epoch: list of (H, K)
    返回: (E, H, K)
    """
    assert len(svals_per_epoch) > 0
    H, K = svals_per_epoch[0].shape
    for sv in svals_per_epoch:
        assert sv.shape == (H, K)
    return torch.stack(svals_per_epoch, dim=0)  # (E,H,K)


def plot_svals_heatmap_per_head(
    svals_epochs: torch.Tensor,
    head_idx: int,
    epoch_indices=None,
    normalize_mode: str = "sum",
    cmap: str = "viridis",
    save_path: str = None,
    title: str = None,
):
    """
    画某一个 head 的 singular value 热力图。

    svals_epochs: (E, H, K)
    """
    E, H, K = svals_epochs.shape
    assert 0 <= head_idx < H

    svals_head = svals_epochs[:, head_idx, :]  # (E, K)
    svals_head = normalize_singular_values(svals_head, mode=normalize_mode)
    data = to_numpy(svals_head).T  # (K, E)

    if epoch_indices is None:
        epoch_indices = list(range(1, E + 1))

    plt.figure(figsize=(6, 4))
    im = plt.imshow(
        data,
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        cmap=cmap,
    )
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.xlabel("Epoch")
    plt.ylabel("Singular value index (1..K)")
    if title is None:
        title = f"Head {head_idx}: singular values heatmap"
    plt.title(title)

    xticks_pos = np.linspace(0, E - 1, num=min(E, 6), dtype=int)
    xticks_lbl = [str(epoch_indices[i]) for i in xticks_pos]
    plt.xticks(ticks=xticks_pos, labels=xticks_lbl)

    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path, dpi=150)
    plt.close()


def plot_svals_heatmap_head_mean(
    svals_epochs: torch.Tensor,
    epoch_indices=None,
    normalize_mode: str = "sum",
    cmap: str = "viridis",
    save_path: str = None,
    title: str = None,
):
    """
    对所有 head 的奇异值先归一化 + 求均值，然后画 head-mean 热力图。

    svals_epochs: (E, H, K)
    """
    E, H, K = svals_epochs.shape

    svals_norm = normalize_singular_values(svals_epochs, mode=normalize_mode)  # (E,H,K)
    svals_mean = svals_norm.mean(dim=1)  # (E,K)
    data = to_numpy(svals_mean).T  # (K,E)

    if epoch_indices is None:
        epoch_indices = list(range(1, E + 1))

    plt.figure(figsize=(6, 4))
    im = plt.imshow(
        data,
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        cmap=cmap,
    )
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.xlabel("Epoch")
    plt.ylabel("Singular value index (1..K)")
    if title is None:
        title = "Head-mean singular values heatmap"
    plt.title(title)

    xticks_pos = np.linspace(0, E - 1, num=min(E, 6), dtype=int)
    xticks_lbl = [str(epoch_indices[i]) for i in xticks_pos]
    plt.xticks(ticks=xticks_pos, labels=xticks_lbl)

    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path, dpi=150)
    plt.close()


def plot_effective_rank_curves(
    svals_epochs: torch.Tensor,
    epoch_indices=None,
    save_path: str = None,
    title: str = None,
    show_per_head: bool = True,
    show_mean: bool = True,
):
    """
    画 effective rank 随 epoch 的变化曲线。

    svals_epochs: (E,H,K)
    """
    E, H, K = svals_epochs.shape

    if epoch_indices is None:
        epoch_indices = list(range(1, E + 1))

    erank_epochs = compute_effective_rank(svals_epochs)  # (E,H)
    erank_np = to_numpy(erank_epochs)  # (E,H)

    plt.figure(figsize=(6, 4))

    if show_per_head:
        for h in range(H):
            # 只给第一条线加 label，避免 legend 太乱
            lbl = f"head {h}" if h == 0 else None
            plt.plot(epoch_indices, erank_np[:, h], alpha=0.3, linewidth=1.0, label=lbl)

    if show_mean:
        mean_erank = erank_np.mean(axis=1)
        std_erank = erank_np.std(axis=1)
        plt.plot(epoch_indices, mean_erank, color="black", linewidth=2.0, label="mean erank")
        if H > 1:
            plt.fill_between(
                epoch_indices,
                mean_erank - std_erank,
                mean_erank + std_erank,
                color="gray",
                alpha=0.2,
                label="±1 std",
            )

    plt.xlabel("Epoch")
    plt.ylabel("Effective rank")
    if title is None:
        title = "Effective rank over epochs"
    plt.title(title)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    if save_path is not None:
        plt.savefig(save_path, dpi=150)
    plt.close()

# ============================================================
#  2. 从 ViT 指定 block 抓 Q/K → 奇异值
# ============================================================


# ============================================================
#  3. 训练 + 谱收集：单个实验 (MHA or TPA)
# ============================================================
def _collect_block_svals_one_epoch(
    model,
    val_loader,
    block_idx: int,
    top_k: int,
    num_batches_spec: int,
    device: str,
):
    """
    返回四个：(svals_qk, svals_q, svals_k, svals_v)
      - svals_qk: pre-softmax logits (QK^T) 的奇异值 (H, Kqk)
      - svals_q/k/v: Q/K/V (N x Dh) 的奇异值 (H, Kqkv)
    说明：SVD 放 CPU 上算，避免训练时 GPU 内存紧张导致 OOM。
    """
    model.eval()
    attn_mod = model.vit.blocks[block_idx].attn

    cached_x = {}

    def _hook(module, inputs, output):
        cached_x["x"] = inputs[0].detach()

    handle = attn_mod.register_forward_hook(_hook)

    qk_batches = []
    q_batches  = []
    k_batches  = []
    v_batches  = []
    batches_done = 0

    with torch.no_grad():
        for images, _ in val_loader:
            images = images.to(device)
            cached_x.clear()

            _ = model(images)

            if "x" not in cached_x:
                continue

            x = cached_x["x"]  # (B, N, C)
            B, N, C = x.shape

            # -------- 拿 q,k,v --------
            if hasattr(attn_mod, "qkv"):
                # timm MHA
                qkv = attn_mod.qkv(x)  # (B, N, 3*C)
                H = attn_mod.num_heads
                Dh = C // H
                qkv = qkv.reshape(B, N, 3, H, Dh).permute(2, 0, 3, 1, 4)  # (3, B, H, N, Dh)
                q, k, v = qkv[0], qkv[1], qkv[2]  # (B,H,N,Dh)
                scale = attn_mod.scale
            else:
                # TPA 系列
                q, k, v = attn_mod._make_qkv(x)  # (B,H,N,Dh)
                Dh = q.shape[-1]
                scale = attn_mod.scale if hasattr(attn_mod, "scale") else (Dh ** -0.5)

            H = q.shape[1]
            K_qk  = min(top_k, N)          # QK^T 是 N×N
            K_qkv = min(top_k, min(N, Dh)) # Q/K/V 是 N×Dh

            # -------- 1) QK^T 的奇异值 --------
            attn_logits = (q * scale) @ k.transpose(-2, -1)   # (B,H,N,N)
            attn_mean = attn_logits.mean(dim=0)               # (H,N,N)

            svals_qk = torch.zeros(H, K_qk, dtype=torch.float32)
            for h in range(H):
                A = attn_mean[h].float().cpu()
                sv = torch.linalg.svdvals(A)
                svals_qk[h] = sv[:K_qk]

            # -------- 2) Q/K/V 本体的奇异值：对 batch 平均后做 SVD --------
            q_mean = q.mean(dim=0)  # (H,N,Dh)
            k_mean = k.mean(dim=0)
            v_mean = v.mean(dim=0)

            svals_q = torch.zeros(H, K_qkv, dtype=torch.float32)
            svals_k = torch.zeros(H, K_qkv, dtype=torch.float32)
            svals_v = torch.zeros(H, K_qkv, dtype=torch.float32)

            for h in range(H):
                svq = torch.linalg.svdvals(q_mean[h].float().cpu())
                svk = torch.linalg.svdvals(k_mean[h].float().cpu())
                svv = torch.linalg.svdvals(v_mean[h].float().cpu())
                svals_q[h] = svq[:K_qkv]
                svals_k[h] = svk[:K_qkv]
                svals_v[h] = svv[:K_qkv]

            qk_batches.append(svals_qk)
            q_batches.append(svals_q)
            k_batches.append(svals_k)
            v_batches.append(svals_v)

            batches_done += 1
            if batches_done >= num_batches_spec:
                break

    handle.remove()

    if len(qk_batches) == 0:
        raise RuntimeError("在谱分析时没有拿到任何 batch 的 attention。")

    # 对 num_batches_spec 求平均：-> (H,K)
    svals_qk_epoch = torch.stack(qk_batches, dim=0).mean(dim=0)
    svals_q_epoch  = torch.stack(q_batches,  dim=0).mean(dim=0)
    svals_k_epoch  = torch.stack(k_batches,  dim=0).mean(dim=0)
    svals_v_epoch  = torch.stack(v_batches,  dim=0).mean(dim=0)

    return svals_qk_epoch, svals_q_epoch, svals_k_epoch, svals_v_epoch

def compute_singular_values_from_tokens(
    x: torch.Tensor,
    top_k: int = 32,
    batch_agg: str = "mean",
) -> torch.Tensor:
    """
    x: (B, H, N, D_h)  -> 对每个 head 的 (N, D_h) 做 SVD，取 top_k
    返回: (H, K)
    """
    B, H, N, Dh = x.shape
    K = min(top_k, N, Dh)

    s_list = []
    for h in range(H):
        if batch_agg == "first":
            X = x[0, h]  # (N, Dh)
        else:
            X = x[:, h].mean(dim=0)  # (N, Dh)

        svals_full = torch.linalg.svdvals(X.to(torch.float32))  # (min(N,Dh),)
        s_list.append(svals_full[:K])

    return torch.stack(s_list, dim=0)  # (H, K)



def _collect_block_qkv_svals_one_epoch(
    model,
    val_loader,
    block_idx: int,
    top_k: int,
    num_batches_spec: int,
    device: str,
):
    """
    返回四个谱：(H,K)
      - svals_qk: SVD of pre-softmax logits (q*scale)@k^T
      - svals_q : SVD of Q token matrix (N,Dh)
      - svals_k : SVD of K token matrix (N,Dh)
      - svals_v : SVD of V token matrix (N,Dh)
    """
    model.eval()
    attn_mod = model.vit.blocks[block_idx].attn
    cached_x = {}

    def _hook(module, inputs, output):
        cached_x["x"] = inputs[0].detach()

    handle = attn_mod.register_forward_hook(_hook)

    qk_list, q_list, k_list, v_list = [], [], [], []
    batches_done = 0

    with torch.no_grad():
        for images, _ in val_loader:
            images = images.to(device)
            cached_x.clear()
            _ = model(images)

            if "x" not in cached_x:
                continue

            x = cached_x["x"]  # (B,N,C)
            B, N, C = x.shape

            # ---- 拿 q,k,v ----
            if hasattr(attn_mod, "qkv"):
                qkv = attn_mod.qkv(x)
                H = attn_mod.num_heads
                Dh = C // H
                qkv = qkv.reshape(B, N, 3, H, Dh).permute(2, 0, 3, 1, 4)  # (3,B,H,N,Dh)
                q, k, v = qkv[0], qkv[1], qkv[2]
                scale = attn_mod.scale
            else:
                q, k, v = attn_mod._make_qkv(x)  # (B,H,N,Dh)
                Dh = q.shape[-1]
                scale = attn_mod.scale if hasattr(attn_mod, "scale") else (Dh ** -0.5)

            # ---- 统一 K：保证 Q/K/V 都输出同一个 K ----
            Kkeep = min(top_k, N, Dh)

            # 1) QK^T
            attn_logits = (q * scale) @ k.transpose(-2, -1)     # (B,H,N,N)
            attn_mean = attn_logits.mean(dim=0)                 # (H,N,N)
            svals_qk = torch.zeros(q.shape[1], Kkeep, device=attn_mean.device)
            for h in range(q.shape[1]):
                sv = torch.linalg.svdvals(attn_mean[h].to(torch.float32))
                svals_qk[h] = sv[:Kkeep]

            # 2) Q/K/V token matrix
            svals_q = compute_singular_values_from_tokens(q, top_k=Kkeep, batch_agg="mean")  # (H,K)
            svals_k = compute_singular_values_from_tokens(k, top_k=Kkeep, batch_agg="mean")
            svals_v = compute_singular_values_from_tokens(v, top_k=Kkeep, batch_agg="mean")

            qk_list.append(svals_qk)
            q_list.append(svals_q)
            k_list.append(svals_k)
            v_list.append(svals_v)

            batches_done += 1
            if batches_done >= num_batches_spec:
                break

    handle.remove()

    if len(qk_list) == 0:
        raise RuntimeError("在谱分析时没有拿到任何 batch 的 attention。")

    # 平均多个 batch（如果 num_batches_spec>1）
    s_qk = torch.stack(qk_list, dim=0).mean(dim=0)  # (H,K)
    s_q  = torch.stack(q_list,  dim=0).mean(dim=0)
    s_k  = torch.stack(k_list,  dim=0).mean(dim=0)
    s_v  = torch.stack(v_list,  dim=0).mean(dim=0)

    return s_qk, s_q, s_k, s_v

def run_small_spectrum_experiment(
    attn_type: str,
    total_epochs: int,
    block_idx: int,
    top_k: int,
    num_batches_spec: int,
    dataset_name: str,
    model_name: str,
    data_dir: str,
    img_size: int,
    batch_size: int,
    num_workers: int,
    lr: float,
    weight_decay: float,
    rank_q: int,
    rank_k: int,
    rank_v: int,
    device: str,
    mlp_ratio: float = 2.0,
    mlp_on: str="qkv",
    sinter_A: float = 5e-5,
    sinter_omega: float = 1e4,
):
    """
    小规模训练 + 谱分析（每个 epoch 做一次）：

      - attn_type:
          "mha" / "tpa" / "nonlinear_tpa" / "headwise_nonlinear_tpa" / "sinter_tpa"

      - 每个 epoch：
          1) train_one_epoch
          2) eval_one_epoch
          3) 在指定 block 上做谱分析，拿到 top-K singular values

      - 返回:
          model, hist = {
            "train_loss_curve": [...],
            "val_loss_curve":   [...],
            "train_acc_curve":  [...],
            "val_acc_curve":    [...],
            "best_val_acc":     float,

            # 兼容旧逻辑：默认等同于 qk 的谱 (E,H,K)
            "svals_epochs":     Tensor (E, H, K),

            # 新增：分别对 qk / q / k / v 的谱 (E,H,K)
            "svals_qk_epochs":  Tensor (E, H, K),
            "svals_q_epochs":   Tensor (E, H, K) 或 None,
            "svals_k_epochs":   Tensor (E, H, K) 或 None,
            "svals_v_epochs":   Tensor (E, H, K) 或 None,

            "total_params":     int,
            "kv_cost":          float,
            "num_heads":        int,
            "head_dim":         int,
            "mlp_ratio_used":   float 或 None,
          }
    """
    # ========= 数据 =========
    train_loader, val_loader, num_classes = get_loaders(
        dataset_name=dataset_name,
        data_dir=data_dir,
        batch_size=batch_size,
        img_size=img_size,
        num_workers=num_workers,
    )

    # ========= 构建模型 =========
    pretrained = False

    if attn_type == "mha":
        model = ViTClassifier(
            num_classes=num_classes,
            model_name=model_name,
            pretrained=pretrained,
            attn_type="mha",
        ).to(device)

    elif attn_type == "tpa":
        model = ViTClassifier(
            num_classes=num_classes,
            model_name=model_name,
            pretrained=pretrained,
            attn_type="tpa",
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
        ).to(device)

    elif attn_type == "nonlinear_tpa":
        if mlp_ratio is None:
            mlp_ratio = 1.0
        model = ViTClassifier(
            num_classes=num_classes,
            model_name=model_name,
            pretrained=pretrained,
            attn_type="nonlinear_tpa",
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            nonlinear_mlp_hidden_ratio=mlp_ratio,
            nonlinear_mlp_on = mlp_on,
        ).to(device)

    elif attn_type == "headwise_nonlinear_tpa":
        if mlp_ratio is None:
            mlp_ratio = 1.0
        model = ViTClassifier(
            num_classes=num_classes,
            model_name=model_name,
            pretrained=pretrained,
            attn_type="headwise_nonlinear_tpa",
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            nonlinear_mlp_hidden_ratio=mlp_ratio,
            nonlinear_mlp_on = mlp_on,
        ).to(device)

    elif attn_type == "sinter_tpa":
        if mlp_ratio is None:
            mlp_ratio = 1.0
        model = ViTClassifier(
            num_classes=num_classes,
            model_name=model_name,
            pretrained=pretrained,
            attn_type="sinter_tpa",
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            nonlinear_mlp_hidden_ratio=mlp_ratio,
            sinter_A=sinter_A,
            sinter_omega=sinter_omega,
        ).to(device)

    else:
        raise ValueError(
            f"Unknown attn_type: {attn_type}. "
            f"Expected one of ['mha','tpa','nonlinear_tpa','headwise_nonlinear_tpa','sinter_tpa']"
        )

    total_params = sum(p.numel() for p in model.parameters())

    # ========= KV cost（归一到 MHA=1）=========
    first_attn = model.vit.blocks[0].attn
    if hasattr(first_attn, "dim"):
        dim_attn = first_attn.dim
    else:
        dim_attn = first_attn.qkv.in_features

    num_heads = first_attn.num_heads
    head_dim = dim_attn // num_heads

    kv_mha = 2 * num_heads * head_dim  # = 2 * dim

    if attn_type == "mha":
        kv_cost = 1.0
    else:
        # TPA 系列（含 nonlinear/headwise/sinter）KV 估算公式和你之前一致
        kv_tpa = (rank_k + rank_v) * (num_heads + head_dim)
        kv_cost = kv_tpa / kv_mha

    # ========= 训练相关 =========
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    train_loss_curve = []
    val_loss_curve   = []
    train_acc_curve  = []
    val_acc_curve    = []
    best_val_acc     = 0.0

    # 每个 epoch 一份谱： (H,K)
    svals_qk_epochs_list = []
    svals_q_epochs_list  = []
    svals_k_epochs_list  = []
    svals_v_epochs_list  = []

    scaler = None  # 先保持不开 AMP（跟你现在谱实验一致）

    for epoch in range(total_epochs):
        # ---- train ----
        train_loss, train_acc = train_one_epoch(
            model=model,
            loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
            scaler=scaler,
        )

        # ---- val ----
        val_loss, val_acc = eval_one_epoch(
            model=model,
            loader=val_loader,
            criterion=criterion,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
        )

        train_loss_curve.append(train_loss)
        val_loss_curve.append(val_loss)
        train_acc_curve.append(train_acc)
        val_acc_curve.append(val_acc)
        best_val_acc = max(best_val_acc, val_acc)

        print(
            f"[{attn_type}] Epoch {epoch+1}/{total_epochs} | "
            f"train loss: {train_loss:.4f}, train acc: {train_acc:.3f} | "
            f"val loss: {val_loss:.4f}, val acc: {val_acc:.3f}"
        )

        # ---- spectrum ----
        out = _collect_block_svals_one_epoch(
            model=model,
            val_loader=val_loader,
            block_idx=block_idx,
            top_k=top_k,
            num_batches_spec=num_batches_spec,
            device=device,
        )

        # 兼容两种返回：
        # 1) 新版： (svals_qk, svals_q, svals_k, svals_v)
        # 2) 旧版： svals_qk
        if isinstance(out, (tuple, list)) and len(out) == 4:
            svals_qk_epoch, svals_q_epoch, svals_k_epoch, svals_v_epoch = out
        else:
            svals_qk_epoch = out
            svals_q_epoch  = None
            svals_k_epoch  = None
            svals_v_epoch  = None

        svals_qk_epochs_list.append(svals_qk_epoch.detach().cpu())

        if svals_q_epoch is not None:
            svals_q_epochs_list.append(svals_q_epoch.detach().cpu())
        if svals_k_epoch is not None:
            svals_k_epochs_list.append(svals_k_epoch.detach().cpu())
        if svals_v_epoch is not None:
            svals_v_epochs_list.append(svals_v_epoch.detach().cpu())

    # ---- stack over epochs: (E,H,K) ----
    svals_qk_epochs = torch.stack(svals_qk_epochs_list, dim=0)  # (E,H,K)

    svals_q_epochs = torch.stack(svals_q_epochs_list, dim=0) if len(svals_q_epochs_list) > 0 else None
    svals_k_epochs = torch.stack(svals_k_epochs_list, dim=0) if len(svals_k_epochs_list) > 0 else None
    svals_v_epochs = torch.stack(svals_v_epochs_list, dim=0) if len(svals_v_epochs_list) > 0 else None

    # 兼容你后续 compute_effective_rank(svals_epochs)
    svals_epochs = svals_qk_epochs

    hist = {
        "train_loss_curve": train_loss_curve,
        "val_loss_curve":   val_loss_curve,
        "train_acc_curve":  train_acc_curve,
        "val_acc_curve":    val_acc_curve,
        "best_val_acc":     best_val_acc,

        "svals_epochs":     svals_epochs,

        "svals_qk_epochs":  svals_qk_epochs,
        "svals_q_epochs":   svals_q_epochs,
        "svals_k_epochs":   svals_k_epochs,
        "svals_v_epochs":   svals_v_epochs,

        "total_params":     total_params,
        "kv_cost":          kv_cost,
        "num_heads":        num_heads,
        "head_dim":         head_dim,
        "mlp_ratio_used":   mlp_ratio,
        "optimizer_state":  optimizer.state_dict(),
        "last_epoch": total_epochs - 1,
    }

    return model, hist




In [6]:
def train_model_MHA(
    model: nn.Module,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    total_epochs: int,
    scaler=None,
):
    """
    适用于 MHA（或任何普通单阶段模型）的训练函数。

    行为：
      - 连续跑 total_epochs 轮
      - 每轮：
          - 调用 train_one_epoch / eval_one_epoch
          - 记录 train/val loss & acc
      - 返回一个 dict，字段和其他 train_model_* 保持一致：
          {
            "train_loss_curve": [...],
            "val_loss_curve":   [...],
            "train_acc_curve":  [...],
            "val_acc_curve":    [...],
            "best_val_acc":     float,
          }
    """
    model.to(device)

    train_loss_curve = []
    val_loss_curve   = []
    train_acc_curve  = []
    val_acc_curve    = []
    best_val_acc     = 0.0

    use_amp = scaler is not None  # 预留，和其他函数保持一致

    for epoch in range(total_epochs):
        # 一轮训练
        train_loss, train_acc = train_one_epoch(
            model=model,
            loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
            scaler=scaler,
        )

        # 一轮验证
        val_loss, val_acc = eval_one_epoch(
            model=model,
            loader=val_loader,
            criterion=criterion,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
        )

        train_loss_curve.append(train_loss)
        val_loss_curve.append(val_loss)
        train_acc_curve.append(train_acc)
        val_acc_curve.append(val_acc)
        best_val_acc = max(best_val_acc, val_acc)

        print(
            f"[MHA] Epoch {epoch+1}/{total_epochs} | "
            f"train loss: {train_loss:.4f}, train acc: {train_acc:.3f} | "
            f"val loss: {val_loss:.4f}, val acc: {val_acc:.3f}"
        )

    return {
        "train_loss_curve": train_loss_curve,
        "val_loss_curve":   val_loss_curve,
        "train_acc_curve":  train_acc_curve,
        "val_acc_curve":    val_acc_curve,
        "best_val_acc":     best_val_acc,
    }


TPA module

In [7]:
class ContextualCPAttention(nn.Module):
    """
    CP-style Tensor Product Attention，支持两种模式：
    - contextA = True :
        A_q(x), A_k(x), A_v(x) 是 token 因子 (B, N, R) —— contextual A
        B_q, B_k, B_v 是 head 因子 (R, H, D) —— non-contextual B
        Q = sum_r A_q(x)[r] * B_q[r]

    - contextA = False :
        A_q, A_k, A_v 是全局 token 因子 (R,) —— non-contextual A
        B_q(x), B_k(x), B_v(x) 是 head 因子 (B, N, R, H, D) —— contextual B
        Q = sum_r A_q[r] * B_q(x)[r]
    接口和 timm 的 Attention 一致：forward(x, attn_mask=None) -> (B, N, C)
    """
    def __init__(
        self,
        dim,
        num_heads: int = 8,
        rank_q: int = 16,
        rank_k: int = 16,
        rank_v: int = 16,
        qkv_bias: bool = True,
        qk_scale: float = None,
        attn_drop: float = 0.0,
        proj_drop: float = 0.0,
        contextA: bool = True,   # True: contextual A, False: contextual B
    ):
        super().__init__()
        assert dim % num_heads == 0, "dim must be divisible by num_heads"
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads

        self.rank_q = rank_q
        self.rank_k = rank_k
        self.rank_v = rank_v
        self.contextA = contextA

        # ===== 模式一：contextual A, non-contextual B =====
        if self.contextA:
            # token 因子 A_q(x), A_k(x), A_v(x) : (B, N, R)
            self.A_q = nn.Linear(dim, rank_q, bias=qkv_bias)
            self.A_k = nn.Linear(dim, rank_k, bias=qkv_bias)
            self.A_v = nn.Linear(dim, rank_v, bias=qkv_bias)

            # head 因子 B_q, B_k, B_v : (R, H, D) 全局参数
            self.B_q = nn.Parameter(torch.empty(rank_q, num_heads, self.head_dim))
            self.B_k = nn.Parameter(torch.empty(rank_k, num_heads, self.head_dim))
            self.B_v = nn.Parameter(torch.empty(rank_v, num_heads, self.head_dim))

            nn.init.xavier_uniform_(self.B_q.view(rank_q, -1))
            nn.init.xavier_uniform_(self.B_k.view(rank_k, -1))
            nn.init.xavier_uniform_(self.B_v.view(rank_v, -1))

        # ===== 模式二：non-contextual A, contextual B =====
        else:
            # token 因子 A_q, A_k, A_v : (R,) 全局参数
            self.A_q = nn.Parameter(torch.empty(rank_q))
            self.A_k = nn.Parameter(torch.empty(rank_k))
            self.A_v = nn.Parameter(torch.empty(rank_v))

            nn.init.normal_(self.A_q, std=0.02)
            nn.init.normal_(self.A_k, std=0.02)
            nn.init.normal_(self.A_v, std=0.02)

            # head 因子 B_q(x), B_k(x), B_v(x) : (B, N, R, H, D)
            self.B_q = nn.Linear(dim, rank_q * num_heads * self.head_dim, bias=qkv_bias)
            self.B_k = nn.Linear(dim, rank_k * num_heads * self.head_dim, bias=qkv_bias)
            self.B_v = nn.Linear(dim, rank_v * num_heads * self.head_dim, bias=qkv_bias)

        self.scale = qk_scale or (self.head_dim ** -0.5)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def _make_qkv(self, x):
        """
        x: (B, N, C)
        return q, k, v: (B, H, N, D)
        """
        B, N, C = x.shape

        if self.contextA:
            # ---------- contextual A, non-contextual B ----------
            q_tok = self.A_q(x)   # (B, N, R_q)
            k_tok = self.A_k(x)   # (B, N, R_k)
            v_tok = self.A_v(x)   # (B, N, R_v)

            # 外积重构 Q/K/V: bnr, r h d -> b n h d
            q = torch.einsum("bnr,rhd->bnhd", q_tok, self.B_q)
            k = torch.einsum("bnr,rhd->bnhd", k_tok, self.B_k)
            v = torch.einsum("bnr,rhd->bnhd", v_tok, self.B_v)

        else:
            # ---------- non-contextual A, contextual B ----------
            # A: (R,) -> (1,1,R) -> (B,N,R)
            q_tok = self.A_q.view(1, 1, self.rank_q).expand(B, N, -1)  # (B,N,R_q)
            k_tok = self.A_k.view(1, 1, self.rank_k).expand(B, N, -1)  # (B,N,R_k)
            v_tok = self.A_v.view(1, 1, self.rank_v).expand(B, N, -1)  # (B,N,R_v)

            # B(x): (B,N,R,H,D)
            Bq = self.B_q(x).view(B, N, self.rank_q, self.num_heads, self.head_dim)
            Bk = self.B_k(x).view(B, N, self.rank_k, self.num_heads, self.head_dim)
            Bv = self.B_v(x).view(B, N, self.rank_v, self.num_heads, self.head_dim)

            # bnr, bnrhd -> bnhd
            q = torch.einsum("bnr,bnrhd->bnhd", q_tok, Bq)
            k = torch.einsum("bnr,bnrhd->bnhd", k_tok, Bk)
            v = torch.einsum("bnr,bnrhd->bnhd", v_tok, Bv)

        # reshape 成 (B, H, N, D)
        q = q.permute(0, 2, 1, 3)
        k = k.permute(0, 2, 1, 3)
        v = v.permute(0, 2, 1, 3)

        return q, k, v

    def forward(self, x, attn_mask=None):
        """
        x: (B, N, C)
        attn_mask: 兼容 timm 接口，一般 ViT 不会用到
        """
        B, N, C = x.shape

        q, k, v = self._make_qkv(x)  # (B, H, N, D)

        attn = (q * self.scale) @ k.transpose(-2, -1)  # (B, H, N, N)

        if attn_mask is not None:
            attn = attn + attn_mask

        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        out = attn @ v  # (B, H, N, D)
        out = out.transpose(1, 2).reshape(B, N, C)  # (B, N, C)

        out = self.proj(out)
        out = self.proj_drop(out)
        return out


In [8]:
class TPAAttention(nn.Module):
    """
    True Tensor Product Attention with separate ranks R_q, R_k, R_v,
    按原文公式 100% 走「两向量外积」版本：

      对每个 token t：
        A_Q(x_t) ∈ R^{R_q × h},  B_Q(x_t) ∈ R^{R_q × d_h}
        Q_t = (1 / R_q) * A_Q(x_t)^T B_Q(x_t) ∈ R^{h × d_h}

      K, V 同理。
    """

    def __init__(
        self,
        dim,
        num_heads: int = 8,
        rank_q: int = 16,
        rank_k: int = 2,
        rank_v: int = 2,
        qkv_bias: bool = True,
        qk_scale: float = None,
        attn_drop: float = 0.0,
        proj_drop: float = 0.0,
    ):
        super().__init__()
        assert dim % num_heads == 0, "dim must be divisible by num_heads"

        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads

        self.rank_q = rank_q
        self.rank_k = rank_k
        self.rank_v = rank_v

        # ========== A(x): head-dim factors, shape (R * h) ==========
        # 对应论文里的 A_Q(xt) ∈ R^{R_q×h} 展平之后的线性映射
        self.A_q = nn.Linear(dim, rank_q * num_heads, bias=qkv_bias)
        self.A_k = nn.Linear(dim, rank_k * num_heads, bias=qkv_bias)
        self.A_v = nn.Linear(dim, rank_v * num_heads, bias=qkv_bias)

        # ========== B(x): token-dim factors, shape (R * d_h) ==========
        # 对应论文里的 B_Q(xt) ∈ R^{R_q×d_h} 展平之后的线性映射
        self.B_q = nn.Linear(dim, rank_q * self.head_dim, bias=qkv_bias)
        self.B_k = nn.Linear(dim, rank_k * self.head_dim, bias=qkv_bias)
        self.B_v = nn.Linear(dim, rank_v * self.head_dim, bias=qkv_bias)

        # 缩放和输出
        self.scale = qk_scale or (self.head_dim ** -0.5)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def _make_qkv(self, x):
        """
        x: (B, N, C = dim)

        返回:
          q, k, v: (B, H, N, D_h)
        """
        B, N, C = x.shape
        H, Dh = self.num_heads, self.head_dim

        # ---------- queries ----------
        Aq = self.A_q(x).view(B, N, self.rank_q, H)      # (B,N,R_q,H)
        Bq = self.B_q(x).view(B, N, self.rank_q, Dh)     # (B,N,R_q,D_h)
        # Qt: (B,N,H,D_h) = (1/R_q) * sum_r a_r ⊗ b_r
        Q = torch.einsum("bnrh,bnrd->bnhd", Aq, Bq) / float(self.rank_q)

        # ---------- keys ----------
        Ak = self.A_k(x).view(B, N, self.rank_k, H)      # (B,N,R_k,H)
        Bk = self.B_k(x).view(B, N, self.rank_k, Dh)     # (B,N,R_k,D_h)
        K = torch.einsum("bnrh,bnrd->bnhd", Ak, Bk) / float(self.rank_k)

        # ---------- values ----------
        Av = self.A_v(x).view(B, N, self.rank_v, H)      # (B,N,R_v,H)
        Bv = self.B_v(x).view(B, N, self.rank_v, Dh)     # (B,N,R_v,D_h)
        V = torch.einsum("bnrh,bnrd->bnhd", Av, Bv) / float(self.rank_v)

        # 现在 Q,K,V 形状是 (B, N, H, D_h)，需要转成 (B, H, N, D_h)
        Q = Q.permute(0, 2, 1, 3).contiguous()
        K = K.permute(0, 2, 1, 3).contiguous()
        V = V.permute(0, 2, 1, 3).contiguous()

        return Q, K, V

    def forward(self, x, attn_mask=None):
        """
        x: (B, N, C)
        返回: (B, N, C)
        """
        B, N, C = x.shape
        assert C == self.dim

        q, k, v = self._make_qkv(x)        # (B,H,N,D_h)

        attn = (q * self.scale) @ k.transpose(-2, -1)  # (B,H,N,N)
        if attn_mask is not None:
            attn = attn + attn_mask
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        out = attn @ v                     # (B,H,N,D_h)
        out = out.transpose(1, 2).reshape(B, N, C)  # (B,N,C)

        out = self.proj(out)
        out = self.proj_drop(out)
        return out


def train_model_TPA(
    model: nn.Module,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    total_epochs: int,
    scaler=None,
):
    """
    适用于 TPA（或普通单阶段模型）的训练函数。

    行为：
      - 连续跑 total_epochs 轮
      - 每轮：
          - 调用 train_one_epoch / eval_one_epoch
          - 记录 train/val loss & acc
      - 返回一个 dict，字段和 train_model_two_stage_tucker 保持一致：
          {
            "train_loss_curve": [...],
            "val_loss_curve":   [...],
            "train_acc_curve":  [...],
            "val_acc_curve":    [...],
            "best_val_acc":     float,
          }
    """
    model.to(device)

    train_loss_curve = []
    val_loss_curve   = []
    train_acc_curve  = []
    val_acc_curve    = []
    best_val_acc     = 0.0

    use_amp = scaler is not None

    for epoch in range(total_epochs):
        # 一轮训练
        train_loss, train_acc = train_one_epoch(
            model=model,
            loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
            scaler=scaler,
        )

        # 一轮验证
        val_loss, val_acc = eval_one_epoch(
            model=model,
            loader=val_loader,
            criterion=criterion,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
        )

        train_loss_curve.append(train_loss)
        val_loss_curve.append(val_loss)
        train_acc_curve.append(train_acc)
        val_acc_curve.append(val_acc)
        best_val_acc = max(best_val_acc, val_acc)

        print(
            f"[TPA] Epoch {epoch+1}/{total_epochs} | "
            f"train loss: {train_loss:.4f}, train acc: {train_acc:.3f} | "
            f"val loss: {val_loss:.4f}, val acc: {val_acc:.3f}"
        )

    return {
        "train_loss_curve": train_loss_curve,
        "val_loss_curve":   val_loss_curve,
        "train_acc_curve":  train_acc_curve,
        "val_acc_curve":    val_acc_curve,
        "best_val_acc":     best_val_acc,
    }


In [9]:

import torch
import torch.nn as nn

class NonlinearTPAAttention(nn.Module):
    """
    True TPA + shared MLP 非线性（可选作用于 Q/K/V 的任意组合）
      Q_lin = (1/R_q) * A_Q(x)^T B_Q(x)
      Q = f(Q_lin)

    通过 mlp_on 控制对哪些分支加 MLP：
      - "qkv" / "qk" / "kv" / "q" / "k" / "v" / "none"
    """

    def __init__(
        self,
        dim,
        num_heads: int = 8,
        rank_q: int = 16,
        rank_k: int = 2,
        rank_v: int = 2,
        qkv_bias: bool = True,
        qk_scale: float = None,
        attn_drop: float = 0.0,
        proj_drop: float = 0.0,
        mlp_hidden_ratio: float = 1.0,
        mlp_on: str = "qkv",   # ✅ 新增
    ):
        super().__init__()
        assert dim % num_heads == 0, "dim must be divisible by num_heads"

        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads

        self.rank_q = rank_q
        self.rank_k = rank_k
        self.rank_v = rank_v

        # ====== A(x) / B(x) ======
        self.A_q = nn.Linear(dim, rank_q * num_heads, bias=qkv_bias)
        self.A_k = nn.Linear(dim, rank_k * num_heads, bias=qkv_bias)
        self.A_v = nn.Linear(dim, rank_v * num_heads, bias=qkv_bias)

        self.B_q = nn.Linear(dim, rank_q * self.head_dim, bias=qkv_bias)
        self.B_k = nn.Linear(dim, rank_k * self.head_dim, bias=qkv_bias)
        self.B_v = nn.Linear(dim, rank_v * self.head_dim, bias=qkv_bias)

        # -------- MLP 选择逻辑（和你的 headwise 版本一致的风格）--------
        mlp_on = "none" if mlp_on is None else str(mlp_on).lower().strip()


        self.kv_shared = (mlp_on == "kv_shared")

        allowed = set("qkv")
        if mlp_on in ("", "none", "null", "no", "false", "0"):
            mlp_set = set()

        elif self.kv_shared:
            mlp_set = set("kv")

        else:
            mlp_set = set(mlp_on)
            if not mlp_set.issubset(allowed):
                raise ValueError(f"mlp_on must be subset of 'qkv' or 'none', got: {mlp_on}")
        self.mlp_on = mlp_set

        hidden_dim = int(self.head_dim * mlp_hidden_ratio)

        def make_mlp():
            return nn.Sequential(
                nn.Linear(self.head_dim, hidden_dim),
                nn.GELU(),
                nn.Linear(hidden_dim, self.head_dim),
            )

        # ✅ shared MLP：没选中的分支用 Identity，确保“数值不变”
        self.q_mlp = make_mlp() if "q" in self.mlp_on else nn.Identity()
        if self.kv_shared and ("k" in self.mlp_on) and ("v" in self.mlp_on):
            kv_mlp = make_mlp()
            self.k_mlp = kv_mlp
            self.v_mlp = kv_mlp
        else:
            self.k_mlp = make_mlp() if "k" in self.mlp_on else nn.Identity()
            self.v_mlp = make_mlp() if "v" in self.mlp_on else nn.Identity()

        # attention 输出
        self.scale = qk_scale or (self.head_dim ** -0.5)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def _make_qkv(self, x):
        B, N, C = x.shape
        H, Dh = self.num_heads, self.head_dim

        # ---------- linear TPA ----------
        Aq = self.A_q(x).view(B, N, self.rank_q, H)
        Bq = self.B_q(x).view(B, N, self.rank_q, Dh)
        Q = torch.einsum("bnrh,bnrd->bnhd", Aq, Bq) / float(self.rank_q)

        Ak = self.A_k(x).view(B, N, self.rank_k, H)
        Bk = self.B_k(x).view(B, N, self.rank_k, Dh)
        K = torch.einsum("bnrh,bnrd->bnhd", Ak, Bk) / float(self.rank_k)

        Av = self.A_v(x).view(B, N, self.rank_v, H)
        Bv = self.B_v(x).view(B, N, self.rank_v, Dh)
        V = torch.einsum("bnrh,bnrd->bnhd", Av, Bv) / float(self.rank_v)

        Q = Q.permute(0, 2, 1, 3).contiguous()  # (B,H,N,Dh)
        K = K.permute(0, 2, 1, 3).contiguous()
        V = V.permute(0, 2, 1, 3).contiguous()

        # ---------- optional nonlinearity ----------
        Q = self.q_mlp(Q)
        K = self.k_mlp(K)
        V = self.v_mlp(V)

        return Q, K, V

    def forward(self, x, attn_mask=None):
        B, N, C = x.shape
        assert C == self.dim

        q, k, v = self._make_qkv(x)
        attn = (q * self.scale) @ k.transpose(-2, -1)
        if attn_mask is not None:
            attn = attn + attn_mask
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        out = attn @ v
        out = out.transpose(1, 2).reshape(B, N, C)
        out = self.proj(out)
        out = self.proj_drop(out)
        return out

    


    
def train_model_nonlinear_TPA(
    model: nn.Module,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    total_epochs: int,
    scaler=None,
):
    """
    适用于 NonlinearTPAAttention（或其它单阶段模型）的训练函数。

    行为和 train_model_TPA 一致，只是打印前缀换成 [NonlinearTPA]，
    方便你后面对比两条曲线。
    """
    model.to(device)

    train_loss_curve = []
    val_loss_curve   = []
    train_acc_curve  = []
    val_acc_curve    = []
    best_val_acc     = 0.0

    use_amp = scaler is not None  # 这里先保留这个变量，方便以后需要用

    for epoch in range(total_epochs):
        # 一轮训练
        train_loss, train_acc = train_one_epoch(
            model=model,
            loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
            scaler=scaler,
        )

        # 一轮验证
        val_loss, val_acc = eval_one_epoch(
            model=model,
            loader=val_loader,
            criterion=criterion,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
        )

        train_loss_curve.append(train_loss)
        val_loss_curve.append(val_loss)
        train_acc_curve.append(train_acc)
        val_acc_curve.append(val_acc)
        best_val_acc = max(best_val_acc, val_acc)

        print(
            f"[NonlinearTPA] Epoch {epoch+1}/{total_epochs} | "
            f"train loss: {train_loss:.4f}, train acc: {train_acc:.3f} | "
            f"val loss: {val_loss:.4f}, val acc: {val_acc:.3f}"
        )

    return {
        "train_loss_curve": train_loss_curve,
        "val_loss_curve":   val_loss_curve,
        "train_acc_curve":  train_acc_curve,
        "val_acc_curve":    val_acc_curve,
        "best_val_acc":     best_val_acc,
    }


In [10]:
import torch
import torch.nn as nn


class HeadwiseNonlinearTPAAttention(nn.Module):
    """
    True TPA + 每个 head 独立的 MLP 非线性（可选作用于 Q/K/V 的任意组合）

    通过 mlp_on 控制对哪些分支加 MLP：
      - "qkv" / "qk" / "kv" / "q" / "k" / "v" / "none"
      - 额外支持："kv_shared"
          含义：每个 head 一个 MLP，但该 head 内 K/V 共享同一个 MLP
    """

    def __init__(
        self,
        dim,
        num_heads: int = 8,
        rank_q: int = 16,
        rank_k: int = 2,
        rank_v: int = 2,
        qkv_bias: bool = True,
        qk_scale: float = None,
        attn_drop: float = 0.0,
        proj_drop: float = 0.0,
        mlp_hidden_ratio: float = 1.0,
        mlp_on: str = "qkv",
    ):
        super().__init__()
        assert dim % num_heads == 0, "dim must be divisible by num_heads"

        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads

        self.rank_q = rank_q
        self.rank_k = rank_k
        self.rank_v = rank_v

        # -------- A(x) / B(x) --------
        self.A_q = nn.Linear(dim, rank_q * num_heads, bias=qkv_bias)
        self.A_k = nn.Linear(dim, rank_k * num_heads, bias=qkv_bias)
        self.A_v = nn.Linear(dim, rank_v * num_heads, bias=qkv_bias)

        self.B_q = nn.Linear(dim, rank_q * self.head_dim, bias=qkv_bias)
        self.B_k = nn.Linear(dim, rank_k * self.head_dim, bias=qkv_bias)
        self.B_v = nn.Linear(dim, rank_v * self.head_dim, bias=qkv_bias)

        # -------- MLP 选择逻辑（新增 kv_shared）--------
        mlp_on = "none" if mlp_on is None else str(mlp_on).lower().strip()
        self.kv_shared = (mlp_on == "kv_shared")

        allowed = set("qkv")
        if mlp_on in ("", "none", "null", "no", "false", "0"):
            mlp_set = set()
        elif self.kv_shared:
            mlp_set = set("kv")  # => {"k","v"}
        else:
            mlp_set = set(mlp_on)
            if not mlp_set.issubset(allowed):
                raise ValueError(f"mlp_on must be subset of 'qkv' or 'none' (or 'kv_shared'), got: {mlp_on}")
        self.mlp_on = mlp_set

        hidden_dim = max(1, int(self.head_dim * mlp_hidden_ratio))

        def make_mlp():
            return nn.Sequential(
                nn.Linear(self.head_dim, hidden_dim),
                nn.GELU(),
                nn.Linear(hidden_dim, self.head_dim),
            )

        # -------- 每个 head 一套 MLP；没选中的分支用 Identity --------
        # Q
        if "q" in self.mlp_on:
            self.q_mlps = nn.ModuleList([make_mlp() for _ in range(num_heads)])
        else:
            self.q_mlps = nn.ModuleList([nn.Identity() for _ in range(num_heads)])

        # K/V
        if self.kv_shared:
            # 关键：只注册一次 kv_mlps，避免重复参数引用
            self.kv_mlps = nn.ModuleList([make_mlp() for _ in range(num_heads)])
            self.k_mlps = None
            self.v_mlps = None
        else:
            self.kv_mlps = None
            if "k" in self.mlp_on:
                self.k_mlps = nn.ModuleList([make_mlp() for _ in range(num_heads)])
            else:
                self.k_mlps = nn.ModuleList([nn.Identity() for _ in range(num_heads)])

            if "v" in self.mlp_on:
                self.v_mlps = nn.ModuleList([make_mlp() for _ in range(num_heads)])
            else:
                self.v_mlps = nn.ModuleList([nn.Identity() for _ in range(num_heads)])

        # -------- attention 输出部分 --------
        self.scale = qk_scale or (self.head_dim ** -0.5)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def _apply_headwise_mlp(self, x_bhnd, mlps: nn.ModuleList):
        """
        x_bhnd: (B, H, N, D_h)
        对每个 head 分别过对应的 mlp
        """
        B, H, N, Dh = x_bhnd.shape
        outs = []
        for h in range(H):
            outs.append(mlps[h](x_bhnd[:, h, :, :]))  # (B,N,Dh)
        return torch.stack(outs, dim=1)  # (B,H,N,Dh)

    def _make_qkv(self, x):
        B, N, C = x.shape
        H, Dh = self.num_heads, self.head_dim

        Aq = self.A_q(x).view(B, N, self.rank_q, H)
        Bq = self.B_q(x).view(B, N, self.rank_q, Dh)
        Q = torch.einsum("bnrh,bnrd->bnhd", Aq, Bq) / float(self.rank_q)

        Ak = self.A_k(x).view(B, N, self.rank_k, H)
        Bk = self.B_k(x).view(B, N, self.rank_k, Dh)
        K = torch.einsum("bnrh,bnrd->bnhd", Ak, Bk) / float(self.rank_k)

        Av = self.A_v(x).view(B, N, self.rank_v, H)
        Bv = self.B_v(x).view(B, N, self.rank_v, Dh)
        V = torch.einsum("bnrh,bnrd->bnhd", Av, Bv) / float(self.rank_v)

        Q = Q.permute(0, 2, 1, 3).contiguous()  # (B,H,N,Dh)
        K = K.permute(0, 2, 1, 3).contiguous()
        V = V.permute(0, 2, 1, 3).contiguous()

        # head-wise 非线性
        Q = self._apply_headwise_mlp(Q, self.q_mlps)

        if self.kv_shared:
            K = self._apply_headwise_mlp(K, self.kv_mlps)
            V = self._apply_headwise_mlp(V, self.kv_mlps)
        else:
            K = self._apply_headwise_mlp(K, self.k_mlps)
            V = self._apply_headwise_mlp(V, self.v_mlps)

        return Q, K, V

    def forward(self, x, attn_mask=None):
        B, N, C = x.shape
        assert C == self.dim

        q, k, v = self._make_qkv(x)
        attn = (q * self.scale) @ k.transpose(-2, -1)
        if attn_mask is not None:
            attn = attn + attn_mask
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        out = attn @ v
        out = out.transpose(1, 2).reshape(B, N, C)
        out = self.proj(out)
        out = self.proj_drop(out)
        return out


In [11]:



class Sinter(nn.Module):
    """
    Sinter 激活函数：
        Sinter(x) = x + A * x * sin(omega * x)
                  = x * (1 + A * sin(omega * x))

    A 和 omega 按 LoRAN 论文给的量级设置为默认值：
      - A 很小（1e-5 量级），保证整体接近恒等映射；
      - omega 很大（1e4 量级），在输入空间引入高频微扰。
    """

    def __init__(self, A: float = 5e-5, omega: float = 1e4):
        super().__init__()
        self.A = A
        self.omega = omega

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: 任意形状，逐元素作用
        return x + self.A * x * torch.sin(self.omega * x)
        # 等价写法：return x * (1 + self.A * torch.sin(self.omega * x))


class SinterTPAAttention(nn.Module):
    """
    在原始 TPA 的基础上，加一层简单的 MLP + Sinter 非线性：
      Q_lin = (1/R_q) * A_Q(x)^T B_Q(x)
      Q = f_q(Q_lin)

    f_q / f_k / f_v 都是两层 MLP: D_h -> D_h_hidden -> D_h
    激活函数用 Sinter 而不是 GeLU。
    默认 D_h_hidden = D_h（最简单的版本）
    """

    def __init__(
        self,
        dim,
        num_heads: int = 8,
        rank_q: int = 16,
        rank_k: int = 2,
        rank_v: int = 2,
        qkv_bias: bool = True,
        qk_scale: float = None,
        attn_drop: float = 0.0,
        proj_drop: float = 0.0,
        mlp_hidden_ratio: float = 1.0,  # 隐藏层大小 = ratio * head_dim
        # Sinter 超参数
        sinter_A: float = 5e-5,
        sinter_omega: float = 1e4,
    ):
        super().__init__()
        assert dim % num_heads == 0, "dim must be divisible by num_heads"

        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads

        self.rank_q = rank_q
        self.rank_k = rank_k
        self.rank_v = rank_v

        # ========== A(x): head-dim factors, shape (R * h) ==========
        self.A_q = nn.Linear(dim, rank_q * num_heads, bias=qkv_bias)
        self.A_k = nn.Linear(dim, rank_k * num_heads, bias=qkv_bias)
        self.A_v = nn.Linear(dim, rank_v * num_heads, bias=qkv_bias)

        # ========== B(x): token-dim factors, shape (R * d_h) ==========
        self.B_q = nn.Linear(dim, rank_q * self.head_dim, bias=qkv_bias)
        self.B_k = nn.Linear(dim, rank_k * self.head_dim, bias=qkv_bias)
        self.B_v = nn.Linear(dim, rank_v * self.head_dim, bias=qkv_bias)

        # ---------- 在 D_h 维度上做 MLP + Sinter 非线性 ----------
        hidden_dim = int(self.head_dim * mlp_hidden_ratio)

        def make_mlp():
            return nn.Sequential(
                nn.Linear(self.head_dim, hidden_dim),
                Sinter(A=sinter_A, omega=sinter_omega),
                nn.Linear(hidden_dim, self.head_dim),
            )

        # 可以选择 Q/K/V 各自一个 MLP，也可以视需要改为共享
        self.q_mlp = make_mlp()
        self.k_mlp = make_mlp()
        self.v_mlp = make_mlp()

        # 缩放和输出
        self.scale = qk_scale or (self.head_dim ** -0.5)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def _make_qkv(self, x):
        """
        x: (B, N, C = dim)
        返回:
          q, k, v: (B, H, N, D_h)
        """
        B, N, C = x.shape
        H, Dh = self.num_heads, self.head_dim

        # ---------- linear TPA 部分 ----------
        # Q
        Aq = self.A_q(x).view(B, N, self.rank_q, H)      # (B,N,R_q,H)
        Bq = self.B_q(x).view(B, N, self.rank_q, Dh)     # (B,N,R_q,D_h)
        Q = torch.einsum("bnrh,bnrd->bnhd", Aq, Bq) / float(self.rank_q)

        # K
        Ak = self.A_k(x).view(B, N, self.rank_k, H)      # (B,N,R_k,H)
        Bk = self.B_k(x).view(B, N, self.rank_k, Dh)     # (B,N,R_k,D_h)
        K = torch.einsum("bnrh,bnrd->bnhd", Ak, Bk) / float(self.rank_k)

        # V
        Av = self.A_v(x).view(B, N, self.rank_v, H)      # (B,N,R_v,H)
        Bv = self.B_v(x).view(B, N, self.rank_v, Dh)     # (B,N,R_v,D_h)
        V = torch.einsum("bnrh,bnrd->bnhd", Av, Bv) / float(self.rank_v)

        # 现在 Q,K,V 形状是 (B, N, H, D_h)，需要转成 (B, H, N, D_h)
        Q = Q.permute(0, 2, 1, 3).contiguous()  # (B,H,N,D_h)
        K = K.permute(0, 2, 1, 3).contiguous()
        V = V.permute(0, 2, 1, 3).contiguous()

        # ---------- 在 D_h 维度加一层 MLP: f(Q_lin) ----------
        Q = self.q_mlp(Q)   # (B,H,N,D_h)
        K = self.k_mlp(K)
        V = self.v_mlp(V)

        return Q, K, V

    def forward(self, x, attn_mask=None):
        """
        x: (B, N, C)
        返回: (B, N, C)
        """
        B, N, C = x.shape
        assert C == self.dim

        q, k, v = self._make_qkv(x)        # (B,H,N,D_h)

        attn = (q * self.scale) @ k.transpose(-2, -1)  # (B,H,N,N)
        if attn_mask is not None:
            attn = attn + attn_mask
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        out = attn @ v                     # (B,H,N,D_h)
        out = out.transpose(1, 2).reshape(B, N, C)  # (B,N,C)

        out = self.proj(out)
        out = self.proj_drop(out)
        return out
    


def train_model_sinter_TPA(
    model: nn.Module,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    total_epochs: int,
    scaler=None,
):
    """
    适用于 SinterTPAAttention（或其它单阶段模型）的训练函数。

    跟 train_model_nonlinear_TPA 一样，只是打印前缀换成 [SinterTPA]，
    方便对比曲线。
    """
    model.to(device)

    train_loss_curve = []
    val_loss_curve   = []
    train_acc_curve  = []
    val_acc_curve    = []
    best_val_acc     = 0.0

    use_amp = scaler is not None  # 预留，之后如果要改 AMP 逻辑可以用

    for epoch in range(total_epochs):
        # 一轮训练
        train_loss, train_acc = train_one_epoch(
            model=model,
            loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
            scaler=scaler,
        )

        # 一轮验证
        val_loss, val_acc = eval_one_epoch(
            model=model,
            loader=val_loader,
            criterion=criterion,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
        )

        train_loss_curve.append(train_loss)
        val_loss_curve.append(val_loss)
        train_acc_curve.append(train_acc)
        val_acc_curve.append(val_acc)
        best_val_acc = max(best_val_acc, val_acc)

        print(
            f"[SinterTPA] Epoch {epoch+1}/{total_epochs} | "
            f"train loss: {train_loss:.4f}, train acc: {train_acc:.3f} | "
            f"val loss: {val_loss:.4f}, val acc: {val_acc:.3f}"
        )

    return {
        "train_loss_curve": train_loss_curve,
        "val_loss_curve":   val_loss_curve,
        "train_acc_curve":  train_acc_curve,
        "val_acc_curve":    val_acc_curve,
        "best_val_acc":     best_val_acc,
    }


In [12]:


class ContextualTuckerTPAAttention(nn.Module):
    def __init__(
        self,
        dim: int,
        num_heads: int = 8,
        rank_q: int = 16,
        rank_k: int = 2,
        rank_v: int = 2,
        rank_head: int = None,
        rank_channel: int = None,
        qkv_bias: bool = True,
        qk_scale: float = None,
        attn_drop: float = 0.0,
        proj_drop: float = 0.0,
    ):
        super().__init__()
        assert dim % num_heads == 0
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads

        # -------- rank 解析（和你原来的逻辑一样）---------
        if rank_q is None:
            if rank_head is not None:
                rank_q = rank_head
            elif rank_channel is not None:
                rank_q = rank_channel
            else:
                raise ValueError("rank_q is None and no rank_head/rank_channel provided.")
        if rank_k is None:
            rank_k = rank_q
        if rank_v is None:
            rank_v = rank_q

        self.rank_q = int(rank_q)
        self.rank_k = int(rank_k)
        self.rank_v = int(rank_v)

        Rq, Rk, Rv = self.rank_q, self.rank_k, self.rank_v
        H, Dh = self.num_heads, self.head_dim

        # Stage1: 先走 CP / TPA
        self.cp_only = True

        # 缩放因子，和 TPAAttention 一致： /R
        self.scale_q_tucker = 1.0 / float(Rq)
        self.scale_k_tucker = 1.0 / float(Rk)
        self.scale_v_tucker = 1.0 / float(Rv)

        # ====== Q 分支 A/B/G ======
        # 和 TPAAttention 完全相同的映射维度：
        #   A: (B,N,R,H)  ← out_features = R*H
        #   B: (B,N,R,Dh) ← out_features = R*Dh
        self.q_A_proj = nn.Linear(dim, Rq * H,  bias=qkv_bias)
        self.q_B_proj = nn.Linear(dim, Rq * Dh, bias=qkv_bias)
        self.q_G_proj = nn.Linear(dim, Rq * Rq, bias=qkv_bias)

        # ====== K 分支 ======
        self.k_A_proj = nn.Linear(dim, Rk * H,  bias=qkv_bias)
        self.k_B_proj = nn.Linear(dim, Rk * Dh, bias=qkv_bias)
        self.k_G_proj = nn.Linear(dim, Rk * Rk, bias=qkv_bias)

        # ====== V 分支 ======
        self.v_A_proj = nn.Linear(dim, Rv * H,  bias=qkv_bias)
        self.v_B_proj = nn.Linear(dim, Rv * Dh, bias=qkv_bias)
        self.v_G_proj = nn.Linear(dim, Rv * Rv, bias=qkv_bias)

        self.scale = qk_scale or (self.head_dim ** -0.5)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        self._reset_parameters()

    def _reset_parameters(self):
        """
        为了让 Stage1 尽量贴近 TPAAttention：
          - A/B 使用 nn.Linear 的默认初始化（和 TPAAttention 一样）
          - 只对 G 的线性层做特殊 init
        """
        for m in [self.q_G_proj, self.k_G_proj, self.v_G_proj]:
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
            # 初期减小 G 的影响（Stage2 用）
            m.weight.data.mul_(0.1)

    # ====== CP / TPA 分支：和 TPAAttention 完全同构 ======
    @staticmethod
    def _cp_from_factors(A, B, scale: float):
        """
        A: (B, N, R, H)
        B: (B, N, R, D_h)

        Q[b,n,h,d] = (1/R) * Σ_r A[b,n,r,h] * B[b,n,r,d]
        """
        Q = torch.einsum("bnrh,bnrd->bnhd", A, B)  # (B,N,H,Dh)
        Q = Q * scale
        Q = Q.permute(0, 2, 1, 3).contiguous()     # (B,H,N,Dh)
        return Q

    # ====== Tucker 分支：A^T G B 版本（为了兼容 Stage1 的排布） ======
    @staticmethod
    def _tucker_from_factors(A, G, B, scale: float):
        """
        A: (B, N, R, H)   对应 A(x) ∈ R^{R×H}
        G: (B, N, R, R)
        B: (B, N, R, D_h)

        Q = A^T G B ∈ R^{H×D_h}
        """
        # M = A^T G  → (B,N,H,R)
        # M[b,n,h,q] = Σ_p A[b,n,p,h] * G[b,n,p,q]
        M = torch.einsum("bnph,bnpq->bnhq", A, G)

        # Q = M B    → (B,N,H,Dh)
        # Q[b,n,h,d] = Σ_q M[b,n,h,q] * B[b,n,q,d]
        Q = torch.einsum("bnhq,bnqd->bnhd", M, B)

        Q = Q * scale
        Q = Q.permute(0, 2, 1, 3).contiguous()  # (B,H,N,Dh)
        return Q

    def _make_qkv(self, x):
        """
        x: (B, N, C)
        返回 q,k,v: (B, H, N, D_h)
        """
        B, N, C = x.shape
        H, Dh = self.num_heads, self.head_dim
        Rq, Rk, Rv = self.rank_q, self.rank_k, self.rank_v

        # ---- Q ----
        qA = self.q_A_proj(x).view(B, N, Rq, H)   # (B,N,R,H)  ✅ 与 TPAAttention 对齐
        qB = self.q_B_proj(x).view(B, N, Rq, Dh)  # (B,N,R,Dh)
        if self.cp_only:
            q = self._cp_from_factors(qA, qB, self.scale_q_tucker)
        else:
            qG = self.q_G_proj(x).view(B, N, Rq, Rq)
            q = self._tucker_from_factors(qA, qG, qB, self.scale_q_tucker)

        # ---- K ----
        kA = self.k_A_proj(x).view(B, N, Rk, H)
        kB = self.k_B_proj(x).view(B, N, Rk, Dh)
        if self.cp_only:
            k = self._cp_from_factors(kA, kB, self.scale_k_tucker)
        else:
            kG = self.k_G_proj(x).view(B, N, Rk, Rk)
            k = self._tucker_from_factors(kA, kG, kB, self.scale_k_tucker)

        # ---- V ----
        vA = self.v_A_proj(x).view(B, N, Rv, H)
        vB = self.v_B_proj(x).view(B, N, Rv, Dh)
        if self.cp_only:
            v = self._cp_from_factors(vA, vB, self.scale_v_tucker)
        else:
            vG = self.v_G_proj(x).view(B, N, Rv, Rv)
            v = self._tucker_from_factors(vA, vG, vB, self.scale_v_tucker)

        return q, k, v

    def forward(self, x, attn_mask=None):
        B, N, C = x.shape
        assert C == self.dim

        q, k, v = self._make_qkv(x)  # (B,H,N,Dh)

        attn = (q * self.scale) @ k.transpose(-2, -1)  # (B,H,N,N)
        if attn_mask is not None:
            attn = attn + attn_mask

        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        out = attn @ v  # (B,H,N,Dh)
        out = out.transpose(1, 2).reshape(B, N, C)
        out = self.proj(out)
        out = self.proj_drop(out)
        return out



# ===================== 工具函数：切换模式 / 控制参数训练 =====================

def set_tucker_cp_mode(model: nn.Module, cp_only: bool):
    """
    在整个 model 里，把所有 
      的 cp_only 统一设置。

    Phase 1（TPA 阶段）: cp_only = True
    Phase 2（Tucker 阶段）: cp_only = False
    """
    for m in model.modules():
        if isinstance(m, ContextualTuckerTPAAttention):
            m.cp_only = cp_only


def set_tucker_ab_requires_grad(model: nn.Module, requires_grad: bool):
    """
    控制所有 Tucker 注意力里的 A/B 投影是否参与训练。
    即：
      - q_A_proj, q_B_proj
      - k_A_proj, k_B_proj
      - v_A_proj, v_B_proj
    """
    for m in model.modules():
        if isinstance(m, ContextualTuckerTPAAttention):
            for proj in [
                m.q_A_proj, m.q_B_proj,
                m.k_A_proj, m.k_B_proj,
                m.v_A_proj, m.v_B_proj,
            ]:
                for p in proj.parameters():
                    p.requires_grad = requires_grad


def set_tucker_core_requires_grad(model: nn.Module, requires_grad: bool):
    """
    控制所有 Tucker 注意力里的 G 部分是否参与训练：
      - q_G_proj, k_G_proj, v_G_proj
    """
    for m in model.modules():
        if isinstance(m, ContextualTuckerTPAAttention):
            for proj in [m.q_G_proj, m.k_G_proj, m.v_G_proj]:
                for p in proj.parameters():
                    p.requires_grad = requires_grad


def init_tucker_core_as_identity(model: nn.Module):
    """
    在「Tucker 阶段开始」时调用：
      把所有 G 的线性映射初始化成恒等核 I：

        G_Q(x) ≡ I_{R_q},  G_K(x) ≡ I_{R_k},  G_V(x) ≡ I_{R_v}

    实现方式：
      - 把 q_G_proj / k_G_proj / v_G_proj 的 weight 置 0
      - 把 bias 置成 vec(I_R)
      这样对任何 x 都有 G(x)=I，从而保证：

        A(x) G(x) B(x)^T = A(x) B(x)^T

      => Phase2 初始时，函数形式与 Phase1 完全一致（真·等价 TPA）。
    """
    for m in model.modules():
        if isinstance(m, ContextualTuckerTPAAttention):
            with torch.no_grad():
                for proj, R in [
                    (m.q_G_proj, m.rank_q),
                    (m.k_G_proj, m.rank_k),
                    (m.v_G_proj, m.rank_v),
                ]:
                    if proj is None:
                        continue

                    # W = 0
                    proj.weight.zero_()

                    # 偏置存在则置 0，然后写成 vec(I)
                    if proj.bias is not None:
                        proj.bias.zero_()
                        eye = torch.eye(R, device=proj.weight.device, dtype=proj.weight.dtype)
                        proj.bias.copy_(eye.reshape(-1))


# ===================== 两阶段训练函数 =====================

def train_model_two_stage_tucker(
    model: nn.Module,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    total_epochs: int,
    ab_epochs: int = 40,   # 第一阶段：只训 A/B（CP/TPA）
    scaler=None,
    is_tucker_model: bool = False,
):
    """
    通用 train 函数：

      - 对 MHA / 纯 TPA model：is_tucker_model=False
            → 普通训练 total_epochs 轮。

      - 对 Tucker-TPA：is_tucker_model=True
            → 两阶段：
                * epoch < ab_epochs:
                    cp_only = True
                    A/B 可训练，G 不训练   （函数族 = TPA）
                * epoch == ab_epochs:
                    切换到 Tucker：
                      - cp_only = False
                      - 重置 G 为恒等核 I
                      - 冻结 A/B，只训 G
                * epoch > ab_epochs:
                    继续 Tucker 阶段
    """
    train_loss_curve = []
    val_loss_curve   = []
    train_acc_curve  = []
    val_acc_curve    = []
    best_val_acc     = 0.0

    for epoch in range(total_epochs):

        # ====== 阶段切换逻辑 ======
        if is_tucker_model:
            if epoch < ab_epochs:
                # Phase 1: CP/TPA —— 训 A/B，G 不参与训练
                set_tucker_cp_mode(model, cp_only=True)
                set_tucker_ab_requires_grad(model, True)
                set_tucker_core_requires_grad(model, False)
                phase_tag = "TPA-phase(AB)"

            elif epoch == ab_epochs:
                # Phase 2 开始：把 G 初始化为 I，切换到 Tucker 形式，只训 G
                set_tucker_cp_mode(model, cp_only=False)
                set_tucker_ab_requires_grad(model, False)
                set_tucker_core_requires_grad(model, True)
                init_tucker_core_as_identity(model)
                phase_tag = "Tucker-phase(G, init=I)"

            else:
                # Phase 2 后续：继续 Tucker，只训 G
                set_tucker_cp_mode(model, cp_only=False)
                set_tucker_ab_requires_grad(model, False)
                set_tucker_core_requires_grad(model, True)
                phase_tag = "Tucker-phase(G)"
        else:
            # 非 Tucker 模型：普通训练
            phase_tag = "normal"

        # ====== 一轮训练 & 验证 ======
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, device, epoch, total_epochs, scaler
        )
        val_loss, val_acc = eval_one_epoch(
            model, val_loader, criterion, device, epoch, total_epochs
        )

        train_loss_curve.append(train_loss)
        val_loss_curve.append(val_loss)
        train_acc_curve.append(train_acc)
        val_acc_curve.append(val_acc)
        best_val_acc = max(best_val_acc, val_acc)

        print(
            f"[Epoch {epoch+1}/{total_epochs} | phase={phase_tag}] "
            f"train loss: {train_loss:.4f}, train acc: {train_acc:.3f} | "
            f"val loss: {val_loss:.4f}, val acc: {val_acc:.3f}"
        )

    return {
        "train_loss_curve": train_loss_curve,
        "val_loss_curve": val_loss_curve,
        "train_acc_curve": train_acc_curve,
        "val_acc_curve": val_acc_curve,
        "best_val_acc": best_val_acc,
    }




In [13]:
def set_tucker_cp_mode(model: nn.Module, cp_only: bool):
    """
    在整个 model 里，把所有 ContextualTuckerTPAAttention 的 cp_only 统一设置。

    Phase 1（TPA 阶段）: cp_only = True
    Phase 2（Tucker 阶段）: cp_only = False
    """
    for m in model.modules():
        if isinstance(m, ContextualTuckerTPAAttention):
            m.cp_only = cp_only


def set_tucker_ab_requires_grad(model: nn.Module, requires_grad: bool):
    """
    控制所有 Tucker 注意力里的 A/B 投影是否参与训练：
      - q_A_proj, q_B_proj
      - k_A_proj, k_B_proj
      - v_A_proj, v_B_proj
    """
    for m in model.modules():
        if isinstance(m, ContextualTuckerTPAAttention):
            for proj in [
                m.q_A_proj, m.q_B_proj,
                m.k_A_proj, m.k_B_proj,
                m.v_A_proj, m.v_B_proj,
            ]:
                for p in proj.parameters():
                    p.requires_grad = requires_grad


def set_tucker_core_requires_grad(model: nn.Module, requires_grad: bool):
    """
    控制所有 Tucker 注意力里的 G 部分是否参与训练：
      - q_G_proj, k_G_proj, v_G_proj
    """
    for m in model.modules():
        if isinstance(m, ContextualTuckerTPAAttention):
            for proj in [m.q_G_proj, m.k_G_proj, m.v_G_proj]:
                for p in proj.parameters():
                    p.requires_grad = requires_grad


def init_tucker_core_as_identity(model: nn.Module):
    """
    在「Tucker 第二阶段开始」时调用：
      把所有 G 的线性映射初始化成恒等核 I：

        G_Q(x) ≡ I_{R_q},  G_K(x) ≡ I_{R_k},  G_V(x) ≡ I_{R_v}

    做法：
      - weight 全 0
      - bias 写成 vec(I_R)
    这样对任何 x 都有 G(x)=I，从而保证：
        A(x)^T G(x) B(x) = A(x)^T B(x)
      → 第二阶段一开始与第一阶段前向完全等价。
    """
    for m in model.modules():
        if isinstance(m, ContextualTuckerTPAAttention):
            with torch.no_grad():
                for proj, R in [
                    (m.q_G_proj, m.rank_q),
                    (m.k_G_proj, m.rank_k),
                    (m.v_G_proj, m.rank_v),
                ]:
                    if proj is None:
                        continue

                    # W = 0
                    proj.weight.zero_()

                    if proj.bias is not None:
                        proj.bias.zero_()
                        eye = torch.eye(
                            R,
                            device=proj.weight.device,
                            dtype=proj.weight.dtype,
                        )
                        proj.bias.copy_(eye.reshape(-1))  # vec(I_R)

def train_model_two_stage_tucker(
    model: nn.Module,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    total_epochs: int,
    ab_epochs: int = 40,   # 第一阶段：只训 A/B（CP/TPA）
    scaler=None,
    is_tucker_model: bool = False,
):
    """
    通用两阶段训练函数：

      - 对非 Tucker 模型（例如普通 MHA / 纯 TPA）：
            is_tucker_model = False
        → 完全等同于单阶段训练，跑 total_epochs 轮。

      - 对使用 ContextualTuckerTPAAttention 的模型：
            is_tucker_model = True
        → 两阶段：
            * epoch < ab_epochs:
                - cp_only = True      （纯 TPA / CP）
                - A/B 可训练
                - G 不训练
            * epoch == ab_epochs:
                - cp_only = False     （切换到 Tucker）
                - 初始化 G(x) ≡ I     （前向与上个 epoch 完全等价）
                - 冻结 A/B，只训 G
            * epoch > ab_epochs:
                - cp_only = False
                - A/B 仍冻结
                - 继续只训 G
    """
    model.to(device)

    train_loss_curve = []
    val_loss_curve   = []
    train_acc_curve  = []
    val_acc_curve    = []
    best_val_acc     = 0.0

    for epoch in range(total_epochs):

        # ================== 阶段切换逻辑 ==================
        if is_tucker_model:
            if epoch < ab_epochs:
                # Phase 1: TPA / CP —— 训 A/B，G 不参与训练
                set_tucker_cp_mode(model, cp_only=True)
                set_tucker_ab_requires_grad(model, True)
                set_tucker_core_requires_grad(model, False)
                phase_tag = "TPA-phase(AB)"

            elif epoch == ab_epochs:
                # Phase 2 刚开始：切到 Tucker（A^T G B）
                #   - cp_only=False
                #   - G 初始化为恒等核 I
                #   - 冻结 A/B，只训 G
                set_tucker_cp_mode(model, cp_only=False)
                init_tucker_core_as_identity(model)
                set_tucker_ab_requires_grad(model, False)
                set_tucker_core_requires_grad(model, True)
                phase_tag = "Tucker-phase(G, init=I)"

            else:
                # Phase 2 后续：继续 Tucker，只训 G
                set_tucker_cp_mode(model, cp_only=False)
                set_tucker_ab_requires_grad(model, False)
                set_tucker_core_requires_grad(model, True)
                phase_tag = "Tucker-phase(G)"
        else:
            phase_tag = "normal"

        # ================== 一轮训练 + 验证 ==================
        train_loss, train_acc = train_one_epoch(
            model=model,
            loader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
            scaler=scaler,
        )

        val_loss, val_acc = eval_one_epoch(
            model=model,
            loader=val_loader,
            criterion=criterion,
            device=device,
            epoch=epoch,
            epochs=total_epochs,
        )

        train_loss_curve.append(train_loss)
        val_loss_curve.append(val_loss)
        train_acc_curve.append(train_acc)
        val_acc_curve.append(val_acc)
        best_val_acc = max(best_val_acc, val_acc)

        print(
            f"[Epoch {epoch+1}/{total_epochs} | phase={phase_tag}] "
            f"train loss: {train_loss:.4f}, train acc: {train_acc:.3f} | "
            f"val loss: {val_loss:.4f}, val acc: {val_acc:.3f}"
        )

    return {
        "train_loss_curve": train_loss_curve,
        "val_loss_curve":   val_loss_curve,
        "train_acc_curve":  train_acc_curve,
        "val_acc_curve":    val_acc_curve,
        "best_val_acc":     best_val_acc,
    }


In [14]:
import math
import torch
import torch.nn as nn


class StaticCoreTuckerTPAAttention(nn.Module):
    """
    Tucker-style Tensor Product Attention with:
      - token-dependent A_t(x), B_t(x)
      - *static* global cores G_q, G_k, G_v (non-contextual)

    对每个 token t:
        Q_t = A_t(x) · G_q_eff · B_t(x)^T

    其中 G_q_eff 可以通过 tucker_lambda 在 diag(G) 和 full(G) 之间渐进过渡：
        G_eff = λ * diag(G_raw) + (1-λ) * G_raw
      - λ = 1 → CP/TPA 风格 (只用 diag)
      - λ = 0 → full Tucker
    """

    def __init__(
        self,
        dim: int,
        num_heads: int = 8,
        rank_head: int = 2,     # R_h
        rank_channel: int = 4,  # R_d
        qkv_bias: bool = True,
        qk_scale: float = None,
        attn_drop: float = 0.0,
        proj_drop: float = 0.0,
        progressive_g: bool = True,
        init_lambda: float = 1.0,   # 默认一开始 λ=1：CP/TPA 模式
    ):
        super().__init__()
        assert dim % num_heads == 0, "dim must be divisible by num_heads"
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads

        self.rank_head = rank_head
        self.rank_channel = rank_channel

        Rh, Rd = rank_head, rank_channel
        H, Dh = num_heads, self.head_dim

        # λ ∈ [0,1] : 1 → diag, 0 → full
        self.progressive_g = progressive_g
        self.tucker_lambda = float(init_lambda)

        # Tucker 输出缩放，防止 Q/K 范数随 rank 爆炸
        self.tucker_scale = 1.0 / math.sqrt(rank_head * rank_channel)

        # ---- Q 的 A/B (token-dependent) ----
        self.q_A_proj = nn.Linear(dim, H * Rh, bias=qkv_bias)   # (B,N,H,Rh)
        self.q_B_proj = nn.Linear(dim, Dh * Rd, bias=qkv_bias)  # (B,N,Dh,Rd)
        # ---- Q 的 static core G_q ----
        self.G_q = nn.Parameter(torch.randn(Rh, Rd) * 0.02)

        # ---- K 的 A/B + static core G_k ----
        self.k_A_proj = nn.Linear(dim, H * Rh, bias=qkv_bias)
        self.k_B_proj = nn.Linear(dim, Dh * Rd, bias=qkv_bias)
        self.G_k = nn.Parameter(torch.randn(Rh, Rd) * 0.02)

        # ---- V 的 A/B + static core G_v ----
        self.v_A_proj = nn.Linear(dim, H * Rh, bias=qkv_bias)
        self.v_B_proj = nn.Linear(dim, Dh * Rd, bias=qkv_bias)
        self.G_v = nn.Parameter(torch.randn(Rh, Rd) * 0.02)

        # 标准 attention 组件
        self.scale = qk_scale or (self.head_dim ** -0.5)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        self._reset_parameters()

    def _reset_parameters(self):
        # A/B 全部 xavier，bias 置 0
        for m in [
            self.q_A_proj, self.q_B_proj,
            self.k_A_proj, self.k_B_proj,
            self.v_A_proj, self.v_B_proj,
        ]:
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)

        # static G 用较小 init，避免一开始爆
        for G in [self.G_q, self.G_k, self.G_v]:
            nn.init.xavier_uniform_(G)
            G.data.mul_(0.1)

    # ========== 关键：对 static G 做 λ 控制 ==========

    def _core_with_lambda(self, G_raw: torch.Tensor) -> torch.Tensor:
        """
        G_raw: (R_h, R_d)

        返回 G_eff:
          - 如果 progressive_g=False 或 lambda<=0: 返回 full G
          - 如果 lambda>=1: 只保留 diag(G_raw)
          - 否则: λ*diag(G_raw) + (1-λ)*G_raw
        """
        if (not self.progressive_g) or self.tucker_lambda <= 0.0:
            return G_raw

        lam = float(self.tucker_lambda)
        lam = max(0.0, min(1.0, lam))
        if lam == 0.0:
            return G_raw

        Rh, Rd = G_raw.shape
        k = min(Rh, Rd)

        diag_matrix = torch.zeros_like(G_raw)
        idx = torch.arange(k, device=G_raw.device)
        diag_matrix[idx, idx] = G_raw[idx, idx]

        G_eff = lam * diag_matrix + (1.0 - lam) * G_raw
        return G_eff

    @staticmethod
    def _tucker_from_factors(A, G, B, scale: float):
        """
        A: (B, N, H, R_h)
        G: (R_h, R_d)      or broadcasted to (B,N,R_h,R_d)
        B: (B, N, D_h, R_d)

        return:
          out: (B, H, N, D_h)
        """
        Bsz, N, H, Rh = A.shape
        _, _, Dh, Rd = B.shape

        if G.dim() == 2:
            # (Rh,Rd) -> (1,1,Rh,Rd) -> (B,N,Rh,Rd)
            Gb = G.view(1, 1, Rh, Rd).expand(Bsz, N, Rh, Rd)
        else:
            Gb = G  # 已经是 (B,N,Rh,Rd)

        # (B,N,H,R_h) × (B,N,R_h,R_d) -> (B,N,H,R_d)
        T = torch.einsum("bnhp,bnpq->bnhq", A, Gb)
        # (B,N,H,R_d) × (B,N,D_h,R_d) -> (B,N,H,D_h)
        Q = torch.einsum("bnhq,bndq->bnhd", T, B)
        Q = Q * scale
        Q = Q.permute(0, 2, 1, 3).contiguous()  # (B,H,N,D_h)
        return Q

    def _make_qkv(self, x):
        """
        x: (B, N, C)
        return q, k, v: (B, H, N, D_h)
        """
        Bsz, N, C = x.shape
        H, Dh = self.num_heads, self.head_dim
        Rh, Rd = self.rank_head, self.rank_channel

        # ========= Q =========
        qA = self.q_A_proj(x).view(Bsz, N, H, Rh)
        qB = self.q_B_proj(x).view(Bsz, N, Dh, Rd)
        Gq_eff = self._core_with_lambda(self.G_q)  # (Rh,Rd)
        q = self._tucker_from_factors(qA, Gq_eff, qB, self.tucker_scale)

        # ========= K =========
        kA = self.k_A_proj(x).view(Bsz, N, H, Rh)
        kB = self.k_B_proj(x).view(Bsz, N, Dh, Rd)
        Gk_eff = self._core_with_lambda(self.G_k)
        k = self._tucker_from_factors(kA, Gk_eff, kB, self.tucker_scale)

        # ========= V =========
        vA = self.v_A_proj(x).view(Bsz, N, H, Rh)
        vB = self.v_B_proj(x).view(Bsz, N, Dh, Rd)
        Gv_eff = self._core_with_lambda(self.G_v)
        v = self._tucker_from_factors(vA, Gv_eff, vB, self.tucker_scale)

        return q, k, v

    def forward(self, x, attn_mask=None):
        Bsz, N, C = x.shape
        assert C == self.dim, f"[StaticCoreTuckerTPAAttention] expected dim={self.dim}, got {C}"

        q, k, v = self._make_qkv(x)  # (B, H, N, D_h)

        attn = (q * self.scale) @ k.transpose(-2, -1)  # (B, H, N, N)
        if attn_mask is not None:
            attn = attn + attn_mask

        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        out = attn @ v  # (B,H,N,D_h)
        out = out.transpose(1, 2).reshape(Bsz, N, C)

        out = self.proj(out)
        out = self.proj_drop(out)
        return out


In [15]:
import math
import torch
import torch.nn as nn

class FocusContextualTPAAttention(nn.Module):
    """
    full + low-rank (TPA) 双分支注意力 + 空间 gate：
    - full 分支：正常 MHA
    - low-rank 分支：True TPA（两个向量的 outer product）：
        Q_t = (1/R) * sum_r a_r(x_t) ⊗ b_r(x_t)
      并支持两种 ablation：
        * contextA = True : contextual A, non-contextual B
        * contextA = False: non-contextual A, contextual B
    - gate: 用 full 分支的 attention + 2D 空间权重，决定每个 token
            更偏 full 还是偏 low-rank
    同时维护 alpha 的 running mean，方便估算 KV cache。
    """
    def __init__(
        self,
        dim,
        num_heads=8,
        rank=16,
        qkv_bias=True,
        qk_scale=None,
        attn_drop=0.0,
        proj_drop=0.0,
        grid_size=(14, 14),  # ViT tiny: (224,16) -> (14,14)
        gamma=1.0,           # 空间邻域加权强度
        sigma=None,          # 高斯宽度
        gate_scale=1.0,      # s_norm 放大多少再过 sigmoid
        contextA=True,       # True: contextual A, False: contextual B
    ):
        super().__init__()
        assert dim % num_heads == 0, "dim 必须能整除 num_heads"
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.rank = rank
        self.contextA = contextA

        self.scale = qk_scale or self.head_dim ** -0.5

        # ===== full 分支：标准 qkv =====
        self.qkv_full = nn.Linear(dim, dim * 3, bias=qkv_bias)

        # ===== low-rank 分支：True TPA（两向量 outer product）=====
        H = self.num_heads
        Dh = self.head_dim
        R = self.rank

        if self.contextA:
            # ------- contextual A(x), non-contextual B -------
            # A_q/A_k/A_v(x): (B,N,R*H) -> (B,N,R,H)
            self.A_q = nn.Linear(dim, R * H, bias=qkv_bias)
            self.A_k = nn.Linear(dim, R * H, bias=qkv_bias)
            self.A_v = nn.Linear(dim, R * H, bias=qkv_bias)

            # B_q/B_k/B_v: (R,D_h) 全局参数（non-contextual）
            self.B_q = nn.Parameter(torch.empty(R, Dh))
            self.B_k = nn.Parameter(torch.empty(R, Dh))
            self.B_v = nn.Parameter(torch.empty(R, Dh))

            nn.init.xavier_uniform_(self.B_q)
            nn.init.xavier_uniform_(self.B_k)
            nn.init.xavier_uniform_(self.B_v)

        else:
            # ------- non-contextual A, contextual B(x) -------
            # A_q/A_k/A_v: (R,H) 全局参数
            self.A_q = nn.Parameter(torch.empty(R, H))
            self.A_k = nn.Parameter(torch.empty(R, H))
            self.A_v = nn.Parameter(torch.empty(R, H))

            nn.init.xavier_uniform_(self.A_q)
            nn.init.xavier_uniform_(self.A_k)
            nn.init.xavier_uniform_(self.A_v)

            # B_q/B_k/B_v(x): (B,N,R*D_h) -> (B,N,R,D_h)
            self.B_q = nn.Linear(dim, R * Dh, bias=qkv_bias)
            self.B_k = nn.Linear(dim, R * Dh, bias=qkv_bias)
            self.B_v = nn.Linear(dim, R * Dh, bias=qkv_bias)

        # 输出投影（和原来一样）
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        # ===== 空间 gate（保持原样） =====
        self.grid_size = grid_size
        Hp, Wp = grid_size
        num_patches = Hp * Wp
        self.num_patches = num_patches
        self.gamma = gamma
        self.gate_scale = gate_scale

        # 预计算 2D 距离矩阵 -> 空间权重 w_ij
        coords = []
        for y in range(Hp):
            for x in range(Wp):
                coords.append((x, y))
        coords = torch.tensor(coords, dtype=torch.float32)  # (P,2)
        dists = torch.cdist(coords, coords, p=2)            # (P,P)

        if sigma is None:
            sigma = max(Hp, Wp) / 2.0

        w = 1.0 + gamma * torch.exp(- (dists ** 2) / (2 * sigma ** 2))  # (P,P)
        self.register_buffer("spatial_weight", w)

        self.eps = 1e-6
        self.last_alpha = None

        # 统计 alpha 的 running mean（用来估算 KV cache）
        self.register_buffer("alpha_running_sum", torch.zeros(1))
        self.register_buffer("alpha_count", torch.zeros(1))

    # === low-rank 分支：构造 Q/K/V（True TPA） ===
    def _low_rank_qkv(self, x):
        """
        x: (B, N, C)
        return q, k, v: (B, H, N, D_h)
        """
        B, N, C = x.shape
        H, Dh, R = self.num_heads, self.head_dim, self.rank

        if self.contextA:
            # ---------- contextual A(x) , non-contextual B ----------
            # A(x): (B,N,R,H)
            Aq = self.A_q(x).view(B, N, R, H)
            Ak = self.A_k(x).view(B, N, R, H)
            Av = self.A_v(x).view(B, N, R, H)

            # B: (R,D_h)
            Bq = self.B_q               # (R,D_h)
            Bk = self.B_k
            Bv = self.B_v

            # Q/K/V: (B,N,H,D_h) = (1/R) * sum_r a_r ⊗ b_r
            q = torch.einsum("bnrh,rd->bnhd", Aq, Bq) / float(R)
            k = torch.einsum("bnrh,rd->bnhd", Ak, Bk) / float(R)
            v = torch.einsum("bnrh,rd->bnhd", Av, Bv) / float(R)

        else:
            # ---------- non-contextual A , contextual B(x) ----------
            # A: (R,H)
            Aq = self.A_q              # (R,H)
            Ak = self.A_k
            Av = self.A_v

            # B(x): (B,N,R,D_h)
            Bq = self.B_q(x).view(B, N, R, Dh)
            Bk = self.B_k(x).view(B, N, R, Dh)
            Bv = self.B_v(x).view(B, N, R, Dh)

            # Q/K/V: (B,N,H,D_h) = (1/R) * sum_r a_r ⊗ b_r
            q = torch.einsum("rh,bnrd->bnhd", Aq, Bq) / float(R)
            k = torch.einsum("rh,bnrd->bnhd", Ak, Bk) / float(R)
            v = torch.einsum("rh,bnrd->bnhd", Av, Bv) / float(R)

        # 变成 (B,H,N,D_h)
        q = q.permute(0, 2, 1, 3)
        k = k.permute(0, 2, 1, 3)
        v = v.permute(0, 2, 1, 3)
        return q, k, v

    # === 从 full attention + 2D 位置 得到 gate alpha（保持原样） ===
    def _compute_gate(self, attn_full, gate_strength=1.0):
        """
        attn_full: (B, H, N, N)，已经 softmax
        return alpha_final: (B, N, 1)
        """
        B, H, N, _ = attn_full.shape

        # 对 head 平均: (B,N,N)
        attn_mean = attn_full.mean(dim=1)

        # u[b,i] = sum_j attn[j,i]  (被多少 token 看)
        u = attn_mean.sum(dim=1)  # (B,N)

        if N == self.num_patches + 1:
            has_cls = True
            u_cls = u[:, :1]
            u_patch = u[:, 1:]
        elif N == self.num_patches:
            has_cls = False
            u_patch = u
            u_cls = None
        else:
            # 防御式：假定前面有一些非 patch token
            has_cls = True
            extra = N - self.num_patches
            u_cls = u[:, :extra]
            u_patch = u[:, extra:]

        # 空间加权: s = u_patch @ W^T
        s_patch = torch.matmul(u_patch, self.spatial_weight.t())  # (B,P)

        mean = s_patch.mean(dim=-1, keepdim=True)
        std = s_patch.std(dim=-1, keepdim=True) + self.eps
        s_norm = (s_patch - mean) / std

        alpha_patch = torch.sigmoid(self.gate_scale * s_norm)  # (B,P)

        if has_cls:
            alpha_full = torch.ones(
                B, self.num_patches + 1, 1,
                device=attn_full.device,
                dtype=attn_full.dtype,
            )
            alpha_full[:, 0, 0] = 1.0  # cls 全 full
            alpha_full[:, 1:, 0] = alpha_patch
        else:
            alpha_full = alpha_patch.unsqueeze(-1)  # (B,P,1)

        # gate_strength: 0 -> 全部 1, 1 -> 用 alpha_full
        if gate_strength < 1.0:
            alpha_final = 1.0 - gate_strength * (1.0 - alpha_full)
        else:
            alpha_final = alpha_full

        self.last_alpha = alpha_final.detach()

        # 更新 running mean
        self.alpha_running_sum += alpha_final.detach().sum()
        self.alpha_count += torch.tensor(
            alpha_final.numel(),
            device=alpha_final.device,
            dtype=self.alpha_running_sum.dtype,
        )

        return alpha_final  # (B,N,1)

    def reset_alpha_stats(self):
        self.alpha_running_sum.zero_()
        self.alpha_count.zero_()

    def get_mean_alpha(self):
        if self.alpha_count.item() <= 0:
            return None
        return (self.alpha_running_sum / self.alpha_count).item()

    def forward(self, x, attn_mask=None, gate_strength=1.0):
        B, N, C = x.shape

        # ===== full 分支 =====
        qkv = self.qkv_full(x).reshape(B, N, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # (3,B,H,N,D)
        q_full, k_full, v_full = qkv[0], qkv[1], qkv[2]

        attn_scores_full = (q_full * self.scale) @ k_full.transpose(-2, -1)
        if attn_mask is not None:
            attn_scores_full = attn_scores_full + attn_mask
        attn_full = attn_scores_full.softmax(dim=-1)
        attn_full = self.attn_drop(attn_full)

        # gate
        alpha = self._compute_gate(attn_full, gate_strength=gate_strength)  # (B,N,1)

        out_full = attn_full @ v_full  # (B,H,N,D)
        out_full = out_full.transpose(1, 2).reshape(B, N, C)  # (B,N,C)

        # ===== low-rank 分支（True TPA）=====
        q_low, k_low, v_low = self._low_rank_qkv(x)
        attn_scores_low = (q_low * self.scale) @ k_low.transpose(-2, -1)
        if attn_mask is not None:
            attn_scores_low = attn_scores_low + attn_mask
        attn_low = attn_scores_low.softmax(dim=-1)
        attn_low = self.attn_drop(attn_low)

        out_low = attn_low @ v_low
        out_low = out_low.transpose(1, 2).reshape(B, N, C)

        # ===== gate 混合 =====
        if gate_strength <= 0.0:
            out = out_full
        else:
            out = alpha * out_full + (1.0 - alpha) * out_low

        out = self.proj(out)
        out = self.proj_drop(out)
        return out


In [16]:
# B 也 contextual 的 Focus-TPA（A、B 都是 contextual，两向量 outer product 版本）
import math
import torch
import torch.nn as nn
class FocusTPAAttention(nn.Module):
    """
    full + low-rank (TPA) 双分支注意力 + 空间 gate：
    - full 分支：正常 MHA
    - low-rank 分支：True TPA（两个向量的 outer product）：
        对每个 token t:
          A_Q(x_t) ∈ R^{R × H},  B_Q(x_t) ∈ R^{R × D_h}
          Q_t = (1/R) * sum_r A_Q(x_t)[r] ⊗ B_Q(x_t)[r] ∈ R^{H×D_h}
      K, V 同理，A、B 都是 contextual（依赖 x_t）
    - gate: 用 full 分支的 attention + 2D 空间权重，决定每个 token
            更偏 full 还是偏 low-rank
    同时维护 alpha 的 running mean，方便估算 KV cache。
    """
    def __init__(
        self,
        dim,
        num_heads=8,
        rank=16,
        qkv_bias=True,
        qk_scale=None,
        attn_drop=0.0,
        proj_drop=0.0,
        grid_size=(14, 14),  # ViT tiny: (224,16) -> (14,14)
        gamma=1.0,           # 空间邻域加权强度
        sigma=None,          # 高斯宽度
        gate_scale=1.0,      # s_norm 放大多少再过 sigmoid
    ):
        super().__init__()
        assert dim % num_heads == 0, "dim 必须能整除 num_heads"
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.rank = rank

        self.scale = qk_scale or self.head_dim ** -0.5

        # ===== full 分支：标准 qkv =====
        self.qkv_full = nn.Linear(dim, dim * 3, bias=qkv_bias)

        # ===== low-rank 分支：True TPA（A、B 都 contextual，两个向量 outer product）=====
        H = self.num_heads
        Dh = self.head_dim
        R = self.rank

        # A_q/A_k/A_v(x): (B,N,R*H) -> (B,N,R,H)
        self.A_q = nn.Linear(dim, R * H, bias=qkv_bias)
        self.A_k = nn.Linear(dim, R * H, bias=qkv_bias)
        self.A_v = nn.Linear(dim, R * H, bias=qkv_bias)

        # B_q/B_k/B_v(x): (B,N,R*D_h) -> (B,N,R,D_h)
        self.B_q = nn.Linear(dim, R * Dh, bias=qkv_bias)
        self.B_k = nn.Linear(dim, R * Dh, bias=qkv_bias)
        self.B_v = nn.Linear(dim, R * Dh, bias=qkv_bias)

        # 输出投影
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        # ===== 空间 gate =====
        self.grid_size = grid_size
        Hp, Wp = grid_size
        num_patches = Hp * Wp
        self.num_patches = num_patches
        self.gamma = gamma
        self.gate_scale = gate_scale

        # 预计算 2D 距离矩阵 -> 空间权重 w_ij
        coords = []
        for y in range(Hp):
            for x in range(Wp):
                coords.append((x, y))
        coords = torch.tensor(coords, dtype=torch.float32)  # (P,2)
        dists = torch.cdist(coords, coords, p=2)            # (P,P)

        if sigma is None:
            sigma = max(Hp, Wp) / 2.0

        w = 1.0 + gamma * torch.exp(- (dists ** 2) / (2 * sigma ** 2))  # (P,P)
        self.register_buffer("spatial_weight", w)

        self.eps = 1e-6
        self.last_alpha = None

        # 统计 alpha 的 running mean（用来估算 KV cache）
        self.register_buffer("alpha_running_sum", torch.zeros(1))
        self.register_buffer("alpha_count", torch.zeros(1))

        # 模块级 gate 强度（默认 1.0，训练时会被外部调度）
        self.gate_strength = 1.0

    # === low-rank 分支：构造 Q/K/V（原版 TPA，两向量 outer product） ===
    def _low_rank_qkv(self, x):
        """
        x: (B, N, C)
        return q, k, v: (B, H, N, D)
        """
        B, N, C = x.shape
        H, Dh, R = self.num_heads, self.head_dim, self.rank

        # ---------- Q ----------
        Aq = self.A_q(x).view(B, N, R, H)   # (B,N,R,H)
        Bq = self.B_q(x).view(B, N, R, Dh)  # (B,N,R,D_h)
        # (B,N,H,D_h) = (1/R) * sum_r a_r ⊗ b_r
        q = torch.einsum("bnrh,bnrd->bnhd", Aq, Bq) / float(R)

        # ---------- K ----------
        Ak = self.A_k(x).view(B, N, R, H)
        Bk = self.B_k(x).view(B, N, R, Dh)
        k = torch.einsum("bnrh,bnrd->bnhd", Ak, Bk) / float(R)

        # ---------- V ----------
        Av = self.A_v(x).view(B, N, R, H)
        Bv = self.B_v(x).view(B, N, R, Dh)
        v = torch.einsum("bnrh,bnrd->bnhd", Av, Bv) / float(R)

        # reshape 成 (B, H, N, D)
        q = q.permute(0, 2, 1, 3)  # (B,H,N,D)
        k = k.permute(0, 2, 1, 3)
        v = v.permute(0, 2, 1, 3)
        return q, k, v

    # === 从 full attention + 2D 位置 得到 gate alpha ===
    def _compute_gate(self, attn_full, gate_strength=1.0):
        """
        attn_full: (B, H, N, N)，已经 softmax
        return alpha_final: (B, N, 1)
        """
        B, H, N, _ = attn_full.shape

        # 对 head 平均: (B,N,N)
        attn_mean = attn_full.mean(dim=1)

        # u[b,i] = sum_j attn[j,i]  (被多少 token 看)
        u = attn_mean.sum(dim=1)  # (B,N)

        if N == self.num_patches + 1:
            has_cls = True
            u_cls = u[:, :1]
            u_patch = u[:, 1:]
        elif N == self.num_patches:
            has_cls = False
            u_patch = u
            u_cls = None
        else:
            # 防御式：假定前面有一些非 patch token
            has_cls = True
            extra = N - self.num_patches
            u_cls = u[:, :extra]
            u_patch = u[:, extra:]

        # 空间加权: s = u_patch @ W^T
        s_patch = torch.matmul(u_patch, self.spatial_weight.t())  # (B,P)

        mean = s_patch.mean(dim=-1, keepdim=True)
        std = s_patch.std(dim=-1, keepdim=True) + self.eps
        s_norm = (s_patch - mean) / std

        alpha_patch = torch.sigmoid(self.gate_scale * s_norm)  # (B,P)

        if has_cls:
            alpha_full = torch.ones(
                B, self.num_patches + 1, 1,
                device=attn_full.device,
                dtype=attn_full.dtype,
            )
            alpha_full[:, 0, 0] = 1.0  # cls 全 full
            alpha_full[:, 1:, 0] = alpha_patch
        else:
            alpha_full = alpha_patch.unsqueeze(-1)  # (B,P,1)

        # gate_strength: 0 -> 全部 1, 1 -> 用 alpha_full
        if gate_strength < 1.0:
            alpha_final = 1.0 - gate_strength * (1.0 - alpha_full)
        else:
            alpha_final = alpha_full

        self.last_alpha = alpha_final.detach()

        # 更新 running mean
        self.alpha_running_sum += alpha_final.detach().sum()
        self.alpha_count += torch.tensor(
            alpha_final.numel(),
            device=alpha_final.device,
            dtype=self.alpha_running_sum.dtype,
        )

        return alpha_final  # (B,N,1)

    def reset_alpha_stats(self):
        self.alpha_running_sum.zero_()
        self.alpha_count.zero_()

    def get_mean_alpha(self):
        if self.alpha_count.item() <= 0:
            return None
        return (self.alpha_running_sum / self.alpha_count).item()

    def forward(self, x, attn_mask=None, gate_strength=None):
        B, N, C = x.shape

        # 如果调用方没传 gate_strength，就用模块自己的值（由训练循环控制）
        if gate_strength is None:
            gate_strength = float(self.gate_strength)

        # ===== full 分支 =====
        qkv = self.qkv_full(x).reshape(B, N, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # (3,B,H,N,D)
        q_full, k_full, v_full = qkv[0], qkv[1], qkv[2]

        attn_scores_full = (q_full * self.scale) @ k_full.transpose(-2, -1)
        if attn_mask is not None:
            attn_scores_full = attn_scores_full + attn_mask
        attn_full = attn_scores_full.softmax(dim=-1)
        attn_full = self.attn_drop(attn_full)

        # gate
        alpha = self._compute_gate(attn_full, gate_strength=gate_strength)  # (B,N,1)

        out_full = attn_full @ v_full  # (B,H,N,D)
        out_full = out_full.transpose(1, 2).reshape(B, N, C)  # (B,N,C)

        # ===== low-rank 分支（TPA）=====
        q_low, k_low, v_low = self._low_rank_qkv(x)
        attn_scores_low = (q_low * self.scale) @ k_low.transpose(-2, -1)
        if attn_mask is not None:
            attn_scores_low = attn_scores_low + attn_mask
        attn_low = attn_scores_low.softmax(dim=-1)
        attn_low = self.attn_drop(attn_low)

        out_low = attn_low @ v_low
        out_low = out_low.transpose(1, 2).reshape(B, N, C)

        # ===== gate 混合 =====
        if gate_strength <= 0.0:
            out = out_full
        else:
            out = alpha * out_full + (1.0 - alpha) * out_low

        out = self.proj(out)
        out = self.proj_drop(out)
        return out
def set_focus_tpa_gate_strength(model: nn.Module, gate_strength: float):
    """
    在整个模型里，把所有 FocusTPAAttention 的 gate_strength 设为同一个值。

    gate_strength 含义：
      - 0.0  → out ≈ 纯 full MHA 分支
      - 1.0  → 使用空间 gate：alpha * full + (1-alpha) * low-rank
      - 中间值 → 从纯 full 逐渐过渡到 gated 混合
    """
    from inspect import isclass
    # 防止名字没导入的问题，兼容你之前定义的 FocusTPAAttention
    try:
        FocusClass = FocusTPAAttention
    except NameError:
        raise RuntimeError("请先在上面的 cell 中定义 FocusTPAAttention 类。")

    for m in model.modules():
        if isinstance(m, FocusClass):
            m.gate_strength = float(gate_strength)


def train_model_focus_tpa(
    model: nn.Module,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    total_epochs: int,
    warmup_full_epochs: int = 10,   # 阶段1：纯 full MHA
    ramp_gate_epochs: int = 10,     # 阶段2：gate_strength 线性从 0→1
    scaler=None,
):
    """
    适用于 FocusTPAAttention 的训练函数（full + low-rank True TPA + 空间 gate）。

    三个阶段：
      - Epoch < warmup_full_epochs:
          gate_strength = 0.0       → 输出完全来自 full 分支
      - warmup_full_epochs <= Epoch < warmup_full_epochs + ramp_gate_epochs:
          gate_strength 线性从 0→1 → 逐步引入 low-rank 分支
      - 之后:
          gate_strength = 1.0       → 使用空间 gate 的完整混合
    """
    model.to(device)

    train_loss_curve = []
    val_loss_curve   = []
    train_acc_curve  = []
    val_acc_curve    = []
    best_val_acc     = 0.0

    for epoch in range(total_epochs):
        # ====== 决定本 epoch 的 gate_strength & phase 标记 ======
        if epoch < warmup_full_epochs:
            gate_strength = 0.0
            phase_tag = "focus-warmup(full-only)"
        elif epoch < warmup_full_epochs + ramp_gate_epochs and ramp_gate_epochs > 0:
            # 线性从 0 → 1
            progress = (epoch - warmup_full_epochs + 1) / float(ramp_gate_epochs)
            gate_strength = max(0.0, min(1.0, progress))
            phase_tag = f"focus-ramp(g={gate_strength:.2f})"
        else:
            gate_strength = 1.0
            phase_tag = "focus-mix(full+lowrank)"

        # 把 gate_strength 写进所有 FocusTPAAttention 模块
        set_focus_tpa_gate_strength(model, gate_strength)

        # ====== 一轮训练 & 验证 ======
        # 这里要和你原来 MHA/TPA/Tucker 用的 train_one_epoch / eval_one_epoch 保持一致
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer, device, epoch, total_epochs, scaler
        )
        val_loss, val_acc = eval_one_epoch(
            model, val_loader, criterion, device, epoch, total_epochs
        )

        train_loss_curve.append(train_loss)
        val_loss_curve.append(val_loss)
        train_acc_curve.append(train_acc)
        val_acc_curve.append(val_acc)
        best_val_acc = max(best_val_acc, val_acc)

        print(
            f"[FocusTPA] Epoch {epoch+1}/{total_epochs} | phase={phase_tag} | "
            f"train loss: {train_loss:.4f}, train acc: {train_acc:.3f} | "
            f"val loss: {val_loss:.4f}, val acc: {val_acc:.3f}"
        )

    return {
        "train_loss_curve": train_loss_curve,
        "val_loss_curve":   val_loss_curve,
        "train_acc_curve":  train_acc_curve,
        "val_acc_curve":    val_acc_curve,
        "best_val_acc":     best_val_acc,
    }


In [17]:
from timm.models.vision_transformer import VisionTransformer
import torch.nn as nn

# 记得这几个类已经在别的文件里定义好：
# - TPAAttention
# - ContextualCPAttention   （如果你还用的话）
# - FocusTPAAttention
# - FocusContextualTPAAttention
# - ContextualTuckerTPAAttention
# - StaticCoreTuckerTPAAttention  （如果还需要静态版）


def replace_vit_attn_with_tpa_true(
    vit_model: VisionTransformer,
    rank_q: int,
    rank_k: int,
    rank_v: int,
):
    """
    把 timm ViT 的每个 block.attn 换成：论文原版 True TPA (A/B 都 contextual, 两向量外积)
    使用 TPAAttention（你之前那版实现）。
    """
    for blk in vit_model.blocks:
        old_attn = blk.attn

        dim = old_attn.qkv.in_features
        num_heads = old_attn.num_heads
        attn_drop = float(old_attn.attn_drop.p)
        proj_drop = float(old_attn.proj_drop.p)
        qkv_bias = old_attn.qkv.bias is not None
        qk_scale = old_attn.scale if hasattr(old_attn, "scale") else None

        blk.attn = TPAAttention(
            dim=dim,
            num_heads=num_heads,
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=proj_drop,
        )
def replace_vit_attn_with_nonlinear_tpa(
    vit_model,
    rank_q: int,
    rank_k: int,
    rank_v: int,
    mlp_hidden_ratio: float = 1.0,
    mlp_on: str = "qkv",   # ✅ 新增
):
    for blk in vit_model.blocks:
        old_attn = blk.attn

        dim = old_attn.qkv.in_features
        num_heads = old_attn.num_heads
        attn_drop = float(old_attn.attn_drop.p)
        proj_drop = float(old_attn.proj_drop.p)
        qkv_bias = old_attn.qkv.bias is not None
        qk_scale = old_attn.scale if hasattr(old_attn, "scale") else None

        blk.attn = NonlinearTPAAttention(
            dim=dim,
            num_heads=num_heads,
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=proj_drop,
            mlp_hidden_ratio=mlp_hidden_ratio,
            mlp_on=mlp_on,   # ✅ 关键
        )

        
def replace_vit_attn_with_headwise_nonlinear_tpa(
    vit_model,
    rank_q: int,
    rank_k: int,
    rank_v: int,
    mlp_hidden_ratio: float = 1.0,
    mlp_on: str = "qkv",
):
    
    for blk in vit_model.blocks:
        old_attn = blk.attn

        dim = old_attn.qkv.in_features
        num_heads = old_attn.num_heads
        attn_drop = float(old_attn.attn_drop.p)
        proj_drop = float(old_attn.proj_drop.p)
        qkv_bias = old_attn.qkv.bias is not None
        qk_scale = old_attn.scale if hasattr(old_attn, "scale") else None

        blk.attn = HeadwiseNonlinearTPAAttention(
            dim=dim,
            num_heads=num_heads,
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=proj_drop,
            mlp_hidden_ratio=mlp_hidden_ratio,
            mlp_on=mlp_on,
        )



def replace_vit_attn_with_sinter_tpa(
    vit_model: VisionTransformer,
    rank_q: int,
    rank_k: int,
    rank_v: int,
    mlp_hidden_ratio: float = 1.0,
    sinter_A: float = 5e-5,
    sinter_omega: float = 1e4,
):
    """
    把 timm ViT 的每个 block.attn 换成 SinterTPAAttention 版本的 TPA：
      Q_lin = 1/R * A^T B
      Q = MLP_Sinter(Q_lin)

    参数
    ----
    rank_q, rank_k, rank_v : TPA 的 rank
    mlp_hidden_ratio       : hidden_dim = mlp_hidden_ratio * head_dim
    sinter_A, sinter_omega : Sinter 激活的超参数
    """
    for blk in vit_model.blocks:
        old_attn = blk.attn

        dim = old_attn.qkv.in_features
        num_heads = old_attn.num_heads
        attn_drop = float(old_attn.attn_drop.p)
        proj_drop = float(old_attn.proj_drop.p)
        qkv_bias = old_attn.qkv.bias is not None
        qk_scale = old_attn.scale if hasattr(old_attn, "scale") else None

        blk.attn = SinterTPAAttention(
            dim=dim,
            num_heads=num_heads,
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=proj_drop,
            mlp_hidden_ratio=mlp_hidden_ratio,
            sinter_A=sinter_A,
            sinter_omega=sinter_omega,
        )




def replace_vit_attn_with_contextual_tpa(
    vit_model: VisionTransformer,
    rank_q: int,
    rank_k: int,
    rank_v: int,
    contextA: bool = True,
):
    """
    把 timm ViT 的每个 block.attn 换成：ContextualCPAttention（简化 CP 版 TPA）
      - contextA=True  -> contextual A, non-contextual B
      - contextA=False -> non-contextual A, contextual B
    """
    for blk in vit_model.blocks:
        old_attn = blk.attn

        dim = old_attn.qkv.in_features
        num_heads = old_attn.num_heads
        attn_drop = float(old_attn.attn_drop.p)
        proj_drop = float(old_attn.proj_drop.p)
        qkv_bias = old_attn.qkv.bias is not None
        qk_scale = old_attn.scale if hasattr(old_attn, "scale") else None

        blk.attn = ContextualCPAttention(
            dim=dim,
            num_heads=num_heads,
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=proj_drop,
            contextA=contextA,
        )


def replace_vit_attn_with_focus_tpa(
    vit_model: VisionTransformer,
    rank: int = 2,
    gamma: float = 1.0,
    sigma=None,
    gate_scale: float = 1.0,
):
    """
    把 timm ViT 的每个 block.attn 换成：FocusTPAAttention
    （full MHA + low-rank True TPA + spatial gate）
    """
    # timm 的 ViT 有 patch_embed.grid_size
    grid_size = vit_model.patch_embed.grid_size  # (Hp, Wp)

    for blk in vit_model.blocks:
        old_attn = blk.attn

        dim = old_attn.qkv.in_features
        num_heads = old_attn.num_heads
        attn_drop = float(old_attn.attn_drop.p)
        proj_drop = float(old_attn.proj_drop.p)
        qkv_bias = old_attn.qkv.bias is not None
        qk_scale = old_attn.scale if hasattr(old_attn, "scale") else None

        blk.attn = FocusTPAAttention(
            dim=dim,
            num_heads=num_heads,
            rank=rank,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=proj_drop,
            grid_size=grid_size,
            gamma=gamma,
            sigma=sigma,
            gate_scale=gate_scale,
        )



def replace_vit_attn_with_focus_contextual_tpa(
    vit_model: VisionTransformer,
    rank: int = 2,
    gamma: float = 1.0,
    sigma=None,
    gate_scale: float = 1.0,
    contextA: bool = True,
):
    """
    把 timm ViT 的每个 block.attn 换成：FocusContextualTPAAttention
    （full MHA + contextual TPA + spatial gate）
    """
    grid_size = vit_model.patch_embed.grid_size  # (Hp, Wp)

    for blk in vit_model.blocks:
        old_attn = blk.attn

        dim = old_attn.qkv.in_features
        num_heads = old_attn.num_heads
        attn_drop = float(old_attn.attn_drop.p)
        proj_drop = float(old_attn.proj_drop.p)
        qkv_bias = old_attn.qkv.bias is not None
        qk_scale = old_attn.scale if hasattr(old_attn, "scale") else None

        blk.attn = FocusContextualTPAAttention(
            dim=dim,
            num_heads=num_heads,
            rank=rank,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=proj_drop,
            grid_size=grid_size,
            gamma=gamma,
            sigma=sigma,
            gate_scale=gate_scale,
            contextA=contextA,
        )


# ================== ✅ 重点：新的 Tucker 版本 ==================

def replace_vit_attn_with_contextual_tucker_tpa(
    vit_model: VisionTransformer,
    rank_q: int,
    rank_k: int,
    rank_v: int,
):
    """
    把 timm ViT 的每个 block.attn 换成：ContextualTuckerTPAAttention（统一版）

    接口对齐为 rank_q / rank_k / rank_v，
    这样你可以做到：

        - TPA:    rank_q, rank_k, rank_v
        - Tucker: rank_q, rank_k, rank_v（完全一致）

    并且在两阶段训练里：
      - Phase 1: cp_only=True, 只用 A/B（函数族 = TPA）
      - Phase 2: cp_only=False, 用 A/G/B^T（Tucker 提升 expressive power）
    """
    for blk in vit_model.blocks:
        old_attn = blk.attn

        if not hasattr(old_attn, "qkv"):
            raise ValueError("当前 block.attn 没有 qkv 属性，可能不是标准 timm ViT Attention。")

        dim = old_attn.qkv.in_features
        num_heads = old_attn.num_heads
        attn_drop = float(old_attn.attn_drop.p)
        proj_drop = float(old_attn.proj_drop.p)
        qkv_bias = old_attn.qkv.bias is not None
        qk_scale = getattr(old_attn, "scale", None)

        blk.attn = ContextualTuckerTPAAttention(
            dim=dim,
            num_heads=num_heads,
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=proj_drop,
        )


def replace_vit_attn_with_static_tucker_tpa(
    vit_model: VisionTransformer,
    rank_q: int,
    rank_k: int,
    rank_v: int,
    progressive_g: bool = True,
    init_lambda: float = 1.0,
):
    """
    如果你还保留 StaticCoreTuckerTPAAttention 的话，
    这里也可以按新的 rank_q / rank_k / rank_v 接口改。
    不用的话可以整个函数删掉。
    """
    for blk in vit_model.blocks:
        old_attn = blk.attn

        if hasattr(old_attn, "qkv"):
            dim = old_attn.qkv.in_features
            qkv_bias = old_attn.qkv.bias is not None
        else:
            dim = old_attn.dim
            qkv_bias = True

        num_heads = old_attn.num_heads
        attn_drop = float(old_attn.attn_drop.p)
        proj_drop = float(old_attn.proj_drop.p)
        qk_scale = getattr(old_attn, "scale", None)

        blk.attn = StaticCoreTuckerTPAAttention(
            dim=dim,
            num_heads=num_heads,
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=proj_drop,
            progressive_g=progressive_g,
            init_lambda=init_lambda,
        )


In [18]:

class ViTClassifier(nn.Module):
    def __init__(
        self,
        num_classes,
        model_name: str = "vit_tiny_patch16_224",
        pretrained: bool = False,

        # 选择注意力类型：
        # "mha" / "tpa" / "nonlinear_tpa" / "sinter_tpa" /
        # "contextual_tpa" / "focus_tpa" / "focus_contextual_tpa" / "tucker_tpa"
        attn_type: str = "mha",

        # ====== TPA / Contextual-TPA 的 rank 设置 ======
        rank_q: int = 16,
        rank_k: int = 2,
        rank_v: int = 2,
        contextual_tpa_A: bool = True,   # 对 contextual_tpa: True=contextual A, False=contextual B

        # ====== Nonlinear / Sinter TPA 的 MLP 设置 ======
        nonlinear_mlp_hidden_ratio: float = 2.0,  # hidden_dim = ratio * head_dim

        # Sinter 激活的超参（只在 attn_type == "sinter_tpa" 时使用）
        sinter_A: float = 5e-5,
        sinter_omega: float = 1e4,

        # ====== Focus-(Contextual)-TPA 的参数 ======
        focus_rank: int = 2,
        focus_gamma: float = 1.0,
        focus_gate_scale: float = 1.0,
        focus_contextual_A: bool = True, # 对 focus_contextual_tpa：同上，控制 A/B 谁 contextual

        # ====== Tucker-TPA 的 rank 设置 ======
        # 对于新的 contextual Tucker-TPA：
        #   - 实际只用到 rank_head / rank_channel
        #   - rank_token 在这里可以暂时忽略（兼容原配置）
        rank_token: int = 16,
        rank_head: int = 8,
        rank_channel: int = 8,

        nonlinear_mlp_on: str = "qkv"
    ):
        super().__init__()

        # 创建基础 ViT 模型
        self.vit = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=num_classes,
        )

        # 根据 attn_type 替换 attention
        if attn_type == "mha":
            # 原生 MHA，什么都不做
            pass

        elif attn_type == "tpa":
            # True TPA（A/B 全 contextual，两向量 outer product）
            replace_vit_attn_with_tpa_true(
                self.vit,
                rank_q=rank_q,
                rank_k=rank_k,
                rank_v=rank_v,
            )

        elif attn_type == "nonlinear_tpa":
            # True TPA + head 内一层 MLP 非线性：Q = MLP(1/R A^T B)
            replace_vit_attn_with_nonlinear_tpa(
                self.vit,
                rank_q=rank_q,
                rank_k=rank_k,
                rank_v=rank_v,
                mlp_hidden_ratio=nonlinear_mlp_hidden_ratio,
                mlp_on=nonlinear_mlp_on,
            )
        elif attn_type == "headwise_nonlinear_tpa":
            replace_vit_attn_with_headwise_nonlinear_tpa(
        self.vit,
        rank_q=rank_q,
        rank_k=rank_k,
        rank_v=rank_v,
        mlp_hidden_ratio=nonlinear_mlp_hidden_ratio,
        mlp_on=nonlinear_mlp_on,
    )


        elif attn_type == "sinter_tpa":
            # True TPA + head 内一层 MLP(Sinter)：Q = MLP_Sinter(1/R A^T B)
            replace_vit_attn_with_sinter_tpa(
                self.vit,
                rank_q=rank_q,
                rank_k=rank_k,
                rank_v=rank_v,
                mlp_hidden_ratio=nonlinear_mlp_hidden_ratio,
                sinter_A=sinter_A,
                sinter_omega=sinter_omega,
            )

        elif attn_type == "contextual_tpa":
            # 简化版 Contextual TPA（CP 形式）
            replace_vit_attn_with_contextual_tpa(
                self.vit,
                rank_q=rank_q,
                rank_k=rank_k,
                rank_v=rank_v,
                contextA=contextual_tpa_A,
            )

        elif attn_type == "focus_tpa":
            # Focus-TPA（full + low-rank True TPA）
            replace_vit_attn_with_focus_tpa(
                self.vit,
                rank=focus_rank,
                gamma=focus_gamma,
                sigma=None,
                gate_scale=focus_gate_scale,
            )

        elif attn_type == "focus_contextual_tpa":
            # Focus-Contextual-TPA（full + 低秩 contextual TPA）
            replace_vit_attn_with_focus_contextual_tpa(
                self.vit,
                rank=focus_rank,
                gamma=focus_gamma,
                sigma=None,
                gate_scale=focus_gate_scale,
                contextA=focus_contextual_A,
            )

        elif attn_type == "tucker_tpa":
            # Tucker-style Contextual Tensor Product Attention
            replace_vit_attn_with_contextual_tucker_tpa(
                self.vit,
                rank_q=rank_q,
                rank_k=rank_k,
                rank_v=rank_v,
                # 如果你新版本的函数需要 rank_head / rank_channel，
                # 也可以在这里一起传进去：
                # rank_head=rank_head,
                # rank_channel=rank_channel,
            )

        else:
            raise ValueError(
                f"Unknown attn_type: {attn_type}. "
                f"Expected one of ['mha','tpa','nonlinear_tpa','sinter_tpa',"
                f"'contextual_tpa','focus_tpa','focus_contextual_tpa','tucker_tpa']"
            )

    def forward(self, x):
        return self.vit(x)

In [19]:
# ========= Multi-dataset ablation: CIFAR10 -> CIFAR100 =========
import os
import json
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.optim import AdamW

# ------------------ safety: keep deterministic-ish (if you already have set_global_seed) ------------------
try:
    set_global_seed(GLOBAL_SEED)
except Exception:
    pass

# ===================== 通用配置 =====================
data_dir = "./data"
img_size = 224
num_workers = 2
weight_decay = 0.05
pretrained = False

device = "cuda" if torch.cuda.is_available() else "cpu"
use_amp = False  # 保持你之前的设置

# 模型
model_tag  = "tiny"
model_name = "vit_tiny_patch16_224"

# 优化
batch_size = 128
lr = 3e-4

# ===== epoch & rank（按你要求固定）=====
rank_q, rank_k, rank_v = 16, 2, 2

# ===== 你要求的 ratio 配置（修正 RATIO_MAP）=====
#   KV (indep): 1.5
#   KV_shared: 3.0
#   QKV: 1.0
#   Q/K/V 单独：3.0
RATIO_MAP = {
    "q": 3.0,
    "k": 3.0,
    "v": 3.0,
    "kv": 1.5,
    "kv_shared": 3.0,
    "qkv": 1.0,

    # NEW: head-wise
    "hw_kv": 0.5,
    "hw_kv_shared": 1.0,
    "hw_qkv": 0.33,
}

DATASETS = ["cifar10", "cifar100"]

# ===== 每个数据集的 epoch 数（按你要求：CIFAR10=40, CIFAR100=20）=====
DATASET_EPOCHS = {
    "cifar10": 40,
    "cifar100": 20,
}

# ===== 实验列表（两套数据集一致，按你指定顺序）=====
# 注意：MHA 的 attn_type 这里先写占位符 "__MHA__"，下面会自动探测实际字符串
EXPERIMENT_LIST = [
    {"name": "MHA_baseline",   "attn_type": "__MHA__",         "mlp_on": "none",      "mlp_ratio": 0.0},
    {"name": "TPA_r1622",      "attn_type": "tpa",             "mlp_on": "none",      "mlp_ratio": 0.0},

    {"name": "NonlinearTPA_KV",        "attn_type": "nonlinear_tpa", "mlp_on": "kv",        "mlp_ratio": RATIO_MAP["kv"]},
    {"name": "NonlinearTPA_KV_shared", "attn_type": "nonlinear_tpa", "mlp_on": "kv_shared", "mlp_ratio": RATIO_MAP["kv_shared"]},

    # NEW: head-wise KV variants
    {"name": "NonlinearTPA_HW_KV",        "attn_type": "headwise_nonlinear_tpa", "mlp_on": "qkv",        "mlp_ratio": RATIO_MAP["hw_qkv"]},
    {"name": "NonlinearTPA_HW_KV",        "attn_type": "headwise_nonlinear_tpa", "mlp_on": "kv",        "mlp_ratio": RATIO_MAP["hw_kv"]},
    {"name": "NonlinearTPA_HW_KV_shared", "attn_type": "headwise_nonlinear_tpa", "mlp_on": "kv_shared", "mlp_ratio": RATIO_MAP["hw_kv_shared"]},

    {"name": "NonlinearTPA_QKV",       "attn_type": "nonlinear_tpa", "mlp_on": "qkv",       "mlp_ratio": RATIO_MAP["qkv"]},

    {"name": "NonlinearTPA_Q",         "attn_type": "nonlinear_tpa", "mlp_on": "q",         "mlp_ratio": RATIO_MAP["q"]},
    {"name": "NonlinearTPA_K",         "attn_type": "nonlinear_tpa", "mlp_on": "k",         "mlp_ratio": RATIO_MAP["k"]},
    {"name": "NonlinearTPA_V",         "attn_type": "nonlinear_tpa", "mlp_on": "v",         "mlp_ratio": RATIO_MAP["v"]},
]

EXPERIMENTS = {
    "cifar10":  EXPERIMENT_LIST,
    "cifar100": EXPERIMENT_LIST,
}

# ------------------ helper: top-k val accuracy (for table) ------------------
@torch.no_grad()
def compute_val_topk_acc(model: nn.Module, loader, device: str, k: int = 5) -> float:
    model.eval()
    correct = 0
    total = 0
    for images, targets in loader:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        logits = model(images)
        _, pred = logits.topk(k, dim=1, largest=True, sorted=True)  # (B,k)
        correct += pred.eq(targets.view(-1, 1)).any(dim=1).sum().item()
        total += targets.numel()
    return correct / max(total, 1)

def _ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)
    return path

def _write_json(path: str, obj):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)

def _save_table_csv_md(df: pd.DataFrame, csv_path: str, md_path: str):
    df.to_csv(csv_path, index=False)
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(df.to_markdown(index=False))

def _make_results_table(rows: list, params_tpa_ref: int):
    out = []
    for r in rows:
        params_m = r["params"] / 1e6
        delta_m  = (r["params"] - params_tpa_ref) / 1e6
        out.append({
            "Dataset": r["dataset"],
            "Method":  r["name"],
            "Attn":    r["attn_type"],
            "MLP_on":  r.get("mlp_on", "none"),
            "MLP_ratio": r.get("mlp_ratio", None),
            "MLP_hidden": r.get("mlp_hidden_dim", None),
            "Params(M)": round(params_m, 3),
            "ΔParams vs TPA(M)": round(delta_m, 3),
            "Best epoch": int(r["best_epoch"] + 1),
            "Top-1 Val Acc (best)": round(float(r["best_val_acc"]), 4),
            "Top-5 Val Acc (final)": (None if r.get("val_top5_acc_final") is None else round(float(r["val_top5_acc_final"]), 4)),
        })
    return pd.DataFrame(out)

def _plot_combined_curves(dataset_dir: str, dataset_name: str, rows: list):
    # 1) val acc
    plt.figure()
    for r in rows:
        epochs_range = range(1, len(r["val_acc_curve"]) + 1)
        plt.plot(epochs_range, r["val_acc_curve"], label=r["name"])
    plt.xlabel("Epoch")
    plt.ylabel("Val accuracy")
    plt.title(f"{dataset_name} - {model_tag}: Val accuracy comparison")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dataset_dir, "combined_val_acc.png"))
    plt.close()

    # 2) val loss
    plt.figure()
    for r in rows:
        epochs_range = range(1, len(r["val_loss_curve"]) + 1)
        plt.plot(epochs_range, r["val_loss_curve"], label=r["name"])
    plt.xlabel("Epoch")
    plt.ylabel("Val loss")
    plt.title(f"{dataset_name} - {model_tag}: Val loss comparison")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dataset_dir, "combined_val_loss.png"))
    plt.close()

# ===== 解析 GLOBAL_SEED（用于 run 文件夹命名 & ckpt）=====
try:
    _SEED_TAG = int(GLOBAL_SEED)
except Exception:
    _SEED_TAG = "NA"

# ===== 自动探测 MHA 的 attn_type 字符串（避免你项目里命名不一致导致崩）=====
def _resolve_mha_attn_type(num_classes: int):
    # 你项目里常见命名候选（如不匹配，你可以把你项目里实际用的字符串加到这里）
    candidates = ["mha", "mhsa", "vanilla", "baseline", "standard", "attn"]
    for cand in candidates:
        try:
            m = ViTClassifier(
                num_classes=num_classes,
                model_name=model_name,
                pretrained=pretrained,
                attn_type=cand,
                rank_q=rank_q, rank_k=rank_k, rank_v=rank_v,
            ).to(device)
            del m
            if device == "cuda":
                torch.cuda.empty_cache()
            return cand
        except Exception:
            continue
    raise RuntimeError(
        "Cannot resolve MHA attn_type. Please add the correct string to candidates in _resolve_mha_attn_type()."
    )

# ===================== result 目录 / run 文件夹 =====================
_ensure_dir("result")

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = (
    f"ablation_seed{_SEED_TAG}_{timestamp}_"
    f"{model_tag}_{model_name}_"
    f"EpC10{DATASET_EPOCHS['cifar10']}_C100{DATASET_EPOCHS['cifar100']}_"
    f"bs{batch_size}_lr{lr}_wd{weight_decay}_"
    f"rq{rank_q}_rk{rank_k}_rv{rank_v}_"
    f"ratios_QKV{RATIO_MAP['qkv']}_KV{RATIO_MAP['kv']}_KVsh{RATIO_MAP['kv_shared']}_single{RATIO_MAP['q']}"
)
run_dir = _ensure_dir(os.path.join("result", run_name))
checkpoints_root = _ensure_dir(os.path.join(run_dir, "checkpoints"))
print(f"\n\n######################## Run dir: {run_dir} ########################")

hparams = {
    "datasets": DATASETS,
    "dataset_epochs": DATASET_EPOCHS,
    "model_tag": model_tag,
    "model_name": model_name,
    "img_size": img_size,
    "batch_size": batch_size,
    "lr": lr,
    "weight_decay": weight_decay,
    "pretrained": pretrained,
    "rank_q": rank_q,
    "rank_k": rank_k,
    "rank_v": rank_v,
    "ratio_map": RATIO_MAP,
    "experiments": EXPERIMENTS,
    "timestamp": timestamp,
    "GLOBAL_SEED": _SEED_TAG,
}
_write_json(os.path.join(run_dir, "hparams.json"), hparams)

global_rows = []

# ===================== 主循环：dataset × experiment =====================
for dataset_name in DATASETS:
    total_epochs = DATASET_EPOCHS[dataset_name]
    print(f"\n\n==================== DATASET: {dataset_name} (epochs={total_epochs}) ====================")

    dataset_dir = _ensure_dir(os.path.join(run_dir, dataset_name))
    _write_json(os.path.join(dataset_dir, "hparams.json"), {**hparams, "dataset_name": dataset_name, "total_epochs": total_epochs})

    # 先拿 num_classes
    train_loader, val_loader, num_classes = get_loaders(
        dataset_name=dataset_name,
        data_dir=data_dir,
        batch_size=batch_size,
        img_size=img_size,
        num_workers=num_workers,
    )

    # 解析 MHA 的 attn_type（只做一次）
    mha_attn_type = _resolve_mha_attn_type(num_classes)
    print(f"[Info] Resolved MHA attn_type = {mha_attn_type}")

    # baseline TPA params (per dataset：因为 num_classes 会影响分类头参数)
    tpa_ref_model = ViTClassifier(
        num_classes=num_classes,
        model_name=model_name,
        pretrained=pretrained,
        attn_type="tpa",
        rank_q=rank_q,
        rank_k=rank_k,
        rank_v=rank_v,
    ).to(device)
    params_tpa_ref = sum(p.numel() for p in tpa_ref_model.parameters())
    del tpa_ref_model
    if device == "cuda":
        torch.cuda.empty_cache()

    dataset_rows = []

    for exp in EXPERIMENTS[dataset_name]:
        exp_name  = exp["name"]
        attn_type = exp["attn_type"]
        mlp_on    = exp["mlp_on"]
        mlp_ratio = exp["mlp_ratio"]

        # 替换 MHA 占位符
        if attn_type == "__MHA__":
            attn_type = mha_attn_type

        print(f"\n-------------------- [{dataset_name}] Experiment: {exp_name} --------------------")
        exp_dir = _ensure_dir(os.path.join(dataset_dir, exp_name))
        exp_ckpt_dir = _ensure_dir(os.path.join(exp_dir, "checkpoints"))

        # 训练（严格保持你现成的 run_small_spectrum_experiment 调用方式）
        model, hist = run_small_spectrum_experiment(
            dataset_name=dataset_name,
            num_workers=2,
            model_name=model_name,
            data_dir=data_dir,
            batch_size=batch_size,
            img_size=img_size,
            lr=lr,
            weight_decay=weight_decay,

            device=device,
            attn_type=attn_type,
            total_epochs=total_epochs,
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            mlp_ratio=mlp_ratio,
            mlp_on=mlp_on,
            block_idx=5,
            top_k=8,
            num_batches_spec=1,
        )

        # final top-5
        try:
            val_top5 = compute_val_topk_acc(model, val_loader, device=device, k=5)
        except Exception:
            val_top5 = None

        train_loss_curve = hist["train_loss_curve"]
        val_loss_curve   = hist["val_loss_curve"]
        train_acc_curve  = hist["train_acc_curve"]
        val_acc_curve    = hist["val_acc_curve"]
        best_val_acc     = float(hist["best_val_acc"])
        best_epoch       = int(np.argmax(val_acc_curve))

        total_params = sum(p.numel() for p in model.parameters())
        first_attn = model.vit.blocks[0].attn
        dim_attn = getattr(first_attn, "dim", None) or first_attn.qkv.in_features
        num_heads_attn = first_attn.num_heads
        head_dim_attn = dim_attn // num_heads_attn

        # 只有 nonlinear_tpa 才有 mlp_hidden_dim
        mlp_hidden_dim = None
        if attn_type == "nonlinear_tpa":
            try:
                mlp_hidden_dim = int(head_dim_attn * float(mlp_ratio))
            except Exception:
                mlp_hidden_dim = None

        # log
        log_lines = []
        for ep in range(total_epochs):
            log_lines.append(
                f"Epoch {ep+1}/{total_epochs} | "
                f"train loss: {train_loss_curve[ep]:.4f}, train acc: {train_acc_curve[ep]:.3f} | "
                f"val loss: {val_loss_curve[ep]:.4f}, val acc: {val_acc_curve[ep]:.3f}"
            )
        with open(os.path.join(exp_dir, "log.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(log_lines))

        # ===== checkpoint：保存尽可能完整的状态，支持“等价续训” =====
        ckpt = {
            "model_state": model.state_dict(),
            "optimizer_state": hist.get("optimizer_state", None),
            "scheduler_state": hist.get("scheduler_state", None),  # 如果你的训练函数有返回就会存
            "amp_scaler_state": hist.get("amp_scaler_state", None),  # use_amp=False 时一般为 None

            "last_epoch": hist.get("last_epoch", None),
            "dataset": dataset_name,
            "attn_type": attn_type,
            "exp_name": exp_name,
            "rank_q": rank_q, "rank_k": rank_k, "rank_v": rank_v,
            "mlp_on": mlp_on,
            "mlp_ratio": mlp_ratio,
            "total_epochs": total_epochs,
            "GLOBAL_SEED": _SEED_TAG,

            # RNG states（尽量保证 resume “工程等价”）
            "numpy_rng_state": np.random.get_state(),
            "torch_rng_state": torch.get_rng_state(),
            "cuda_rng_state": (torch.cuda.get_rng_state_all() if torch.cuda.is_available() else None),

            # metrics/history
            "best_val_acc": best_val_acc,
            "train_loss_curve": train_loss_curve,
            "val_loss_curve": val_loss_curve,
            "train_acc_curve": train_acc_curve,
            "val_acc_curve": val_acc_curve,
            "val_top5_acc_final": val_top5,
            "hparams": hparams,

            # 方便复现实验/续训时重建同构调用
            "train_call_kwargs": {
                "dataset_name": dataset_name,
                "num_workers": 2,
                "model_name": model_name,
                "data_dir": data_dir,
                "batch_size": batch_size,
                "img_size": img_size,
                "lr": lr,
                "weight_decay": weight_decay,
                "device": device,
                "attn_type": attn_type,
                "total_epochs": total_epochs,
                "rank_q": rank_q,
                "rank_k": rank_k,
                "rank_v": rank_v,
                "mlp_ratio": mlp_ratio,
                "mlp_on": mlp_on,
                "block_idx": 5,
                "top_k": 8,
                "num_batches_spec": 1,
            }
        }

        # 1) 保持你原逻辑：exp_dir/ckpt.pt
        torch.save(ckpt, os.path.join(exp_dir, "ckpt.pt"))
        # 2) 按你要求：全部也落在 checkpoints 目录里（每个 exp 一份）
        torch.save(ckpt, os.path.join(exp_ckpt_dir, "ckpt.pt"))
        # 3) run 级别汇总 checkpoints（便于你后续脚本统一读取）
        torch.save(ckpt, os.path.join(checkpoints_root, f"{dataset_name}__{exp_name}__seed{_SEED_TAG}.pt"))

        # plots（保持原有逻辑）
        epochs_range = range(1, total_epochs + 1)

        plt.figure()
        plt.plot(epochs_range, train_loss_curve, label="train loss")
        plt.plot(epochs_range, val_loss_curve, label="val loss")
        plt.xlabel("Epoch"); plt.ylabel("Loss")
        plt.title(f"{dataset_name} - {model_tag} - {exp_name}: loss convergence")
        plt.grid(True); plt.legend(); plt.tight_layout()
        plt.savefig(os.path.join(exp_dir, "loss_curve.png"))
        plt.close()

        plt.figure()
        plt.plot(epochs_range, val_acc_curve, label=f"{exp_name} val acc")
        plt.xlabel("Epoch"); plt.ylabel("Val accuracy")
        plt.title(f"{dataset_name} - {model_tag} - {exp_name}: val acc")
        plt.grid(True); plt.legend(); plt.tight_layout()
        plt.savefig(os.path.join(exp_dir, "val_acc_curve.png"))
        plt.close()

        # update rows
        row = {
            "dataset": dataset_name,
            "name": exp_name,
            "attn_type": attn_type,
            "mlp_on": mlp_on,
            "mlp_ratio": (None if attn_type != "nonlinear_tpa" else float(mlp_ratio)),
            "mlp_hidden_dim": mlp_hidden_dim,
            "params": total_params,
            "params_vs_tpa": total_params - params_tpa_ref,
            "best_val_acc": best_val_acc,
            "best_epoch": best_epoch,
            "val_top5_acc_final": val_top5,
            "train_loss_curve": train_loss_curve,
            "val_loss_curve": val_loss_curve,
            "train_acc_curve": train_acc_curve,
            "val_acc_curve": val_acc_curve,
            "GLOBAL_SEED": _SEED_TAG,
        }
        dataset_rows.append(row)
        global_rows.append(row)

        # dataset summary/table/combined plots（每个 experiment 完成就刷新一次，防中途挂）
        dataset_summary = []
        for r in dataset_rows:
            dataset_summary.append({
                k: r[k] for k in [
                    "dataset","name","attn_type","mlp_on","mlp_ratio","mlp_hidden_dim",
                    "params","params_vs_tpa","best_val_acc","best_epoch","val_top5_acc_final",
                    "train_loss_curve","val_loss_curve","train_acc_curve","val_acc_curve","GLOBAL_SEED"
                ]
            })
        _write_json(os.path.join(dataset_dir, "summary.json"), dataset_summary)

        df_dataset = _make_results_table(dataset_rows, params_tpa_ref=params_tpa_ref)
        _save_table_csv_md(
            df_dataset,
            csv_path=os.path.join(dataset_dir, "results_table.csv"),
            md_path=os.path.join(dataset_dir, "results_table.md"),
        )
        _plot_combined_curves(dataset_dir, dataset_name, dataset_rows)

        # global 同步写一份（同样防 crash）
        global_summary = []
        for r in global_rows:
            global_summary.append({
                k: r[k] for k in [
                    "dataset","name","attn_type","mlp_on","mlp_ratio","mlp_hidden_dim",
                    "params","params_vs_tpa","best_val_acc","best_epoch","val_top5_acc_final",
                    "train_loss_curve","val_loss_curve","train_acc_curve","val_acc_curve","GLOBAL_SEED"
                ]
            })
        _write_json(os.path.join(run_dir, "summary.json"), global_summary)

        pd.DataFrame([{
            "Dataset": r["dataset"],
            "Method": r["name"],
            "Attn": r["attn_type"],
            "MLP_on": r.get("mlp_on","none"),
            "MLP_ratio": r.get("mlp_ratio", None),
            "MLP_hidden": r.get("mlp_hidden_dim", None),
            "Params(M)": round(r["params"]/1e6, 3),
            "Top-1 Val Acc (best)": round(float(r["best_val_acc"]), 4),
            "Best epoch": int(r["best_epoch"] + 1),
            "Top-5 Val Acc (final)": (None if r.get("val_top5_acc_final") is None else round(float(r["val_top5_acc_final"]), 4)),
            "GLOBAL_SEED": r.get("GLOBAL_SEED", None),
        } for r in global_rows]).to_csv(os.path.join(run_dir, "results_table.csv"), index=False)




######################## Run dir: result\ablation_seed2_20260105_113812_tiny_vit_tiny_patch16_224_EpC1040_C10020_bs128_lr0.0003_wd0.05_rq16_rk2_rv2_ratios_QKV1.0_KV1.5_KVsh3.0_single3.0 ########################


==================== DATASET: cifar10 (epochs=40) ====================
[Info] Resolved MHA attn_type = mha

-------------------- [cifar10] Experiment: MHA_baseline --------------------


Val   [1/40]: 100%|██████████████████████████████████████| 79/79 [00:10<00:00,  7.41it/s, loss=1.66]


[mha] Epoch 1/40 | train loss: 1.8298, train acc: 0.319 | val loss: 1.6354, val acc: 0.393


Val   [2/40]: 100%|██████████████████████████████████████| 79/79 [00:10<00:00,  7.60it/s, loss=1.42]


[mha] Epoch 2/40 | train loss: 1.5606, train acc: 0.426 | val loss: 1.4687, val acc: 0.467


...

[nonlinear_tpa] Epoch 18/20 | train loss: 0.9090, train acc: 0.731 | val loss: 2.5957, val acc: 0.416


Val   [19/20]: 100%|█████████████████████████████████████| 79/79 [00:18<00:00,  4.23it/s, loss=2.18]


[nonlinear_tpa] Epoch 19/20 | train loss: 0.7692, train acc: 0.770 | val loss: 2.7115, val acc: 0.406


Val   [20/20]: 100%|█████████████████████████████████████| 79/79 [00:18<00:00,  4.22it/s, loss=1.97]


[nonlinear_tpa] Epoch 20/20 | train loss: 0.6333, train acc: 0.809 | val loss: 2.7883, val acc: 0.414


import os
import json
import datetime
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.optim import AdamW

# ==== 通用配置 ====
data_dir = "./data"
img_size = 224
num_workers = 8
weight_decay = 0.05
pretrained = False

device = "cuda" if torch.cuda.is_available() else "cpu"
use_amp = False      # 先关 AMP，等结构都稳定后再考虑开

# 数据集 & 模型
dataset_name = "cifar100"     # 换数据集只改这里
model_tag   = "tiny"
model_name  = "vit_tiny_patch16_224"
batch_size  = 128
lr          = 3e-4

# ====== epoch 设置 ======
epochs_base         = 30   # TPA / NonlinearTPA / SinterTPA 的总 epoch
epochs_mha          = 100  # MHA 的总 epoch
tucker_ab_epochs    = 11   # TuckerTPA 阶段 1：训练 A/B 的 epoch 数
tucker_total_epochs = 20   # TuckerTPA 总 epoch

# ====== rank 设置：共享单一 R ======
R_shared = 16
tucker_rank_head    = R_shared
tucker_rank_channel = R_shared

# TPA / NonlinearTPA / SinterTPA 的 rank 设置
rank_q = 16
rank_k = 2
rank_v = 2

# ====== Sinter 超参数（扰动） ======
sinter_A = 5e-5
sinter_omega = 1e4

# ====== 实验列表 ======
# 顺序：
#   1) SinterTPA (ratio=1.0, 30ep)
#   2) NonlinearTPA (ratio=1.0, 30ep)
#   3) MHA (100ep)
#   4) SinterTPA (ratio=2.0, 30ep)
#   5) NonlinearTPA (ratio=2.0, 30ep)
#   6) SinterTPA (ratio=3.0, 30ep)
#   7) NonlinearTPA (ratio=3.0, 30ep)
experiments = [
    {
        "name": "SinterTPA_ratio1.0",
        "attn_type": "sinter_tpa",
        "total_epochs": epochs_base,
        "mlp_ratio": 1.0,
    },
    {
        "name": "NonlinearTPA_ratio1.0",
        "attn_type": "nonlinear_tpa",
        "total_epochs": epochs_base,
        "mlp_ratio": 1.0,
    },
    {
        "name": "MHA",
        "attn_type": "mha",
        "total_epochs": epochs_mha,
        "mlp_ratio": None,
    },
    {
        "name": "SinterTPA_ratio2.0",
        "attn_type": "sinter_tpa",
        "total_epochs": epochs_base,
        "mlp_ratio": 2.0,
    },
    {
        "name": "NonlinearTPA_ratio2.0",
        "attn_type": "nonlinear_tpa",
        "total_epochs": epochs_base,
        "mlp_ratio": 2.0,
    },
    {
        "name": "SinterTPA_ratio3.0",
        "attn_type": "sinter_tpa",
        "total_epochs": epochs_base,
        "mlp_ratio": 3.0,
    },
    {
        "name": "NonlinearTPA_ratio3.0",
        "attn_type": "nonlinear_tpa",
        "total_epochs": epochs_base,
        "mlp_ratio": 3.0,
    },
]

# ============== 准备 result 目录和本次 run 的文件夹 ==============

os.makedirs("result", exist_ok=True)

run_name = (
    f"{dataset_name}_{model_tag}_{model_name}"
    f"_BaseEp{epochs_base}_MHAEp{epochs_mha}"
    f"_bs{batch_size}_lr{lr}_wd{weight_decay}"
    f"_rq{rank_q}_rk{rank_k}_rv{rank_v}"
    f"_attns_SinterTPA_NonlinearTPA_MHA"
)

run_dir = os.path.join("result", run_name)
os.makedirs(run_dir, exist_ok=True)

print(f"\n\n######################## Run dir: {run_dir} ########################")

# 把本次实验的超参数也存一个 json 方便以后查
tested_mlp_ratios = sorted(
    {e["mlp_ratio"] for e in experiments if e.get("mlp_ratio") is not None}
)

hparams = {
    "dataset_name": dataset_name,
    "model_tag": model_tag,
    "model_name": model_name,
    "epochs_base": epochs_base,
    "epochs_mha": epochs_mha,
    "tucker_total_epochs": tucker_total_epochs,
    "tucker_ab_epochs": tucker_ab_epochs,
    "batch_size": batch_size,
    "lr": lr,
    "weight_decay": weight_decay,
    "R_shared": R_shared,
    "rank_q": rank_q,
    "rank_k": rank_k,
    "rank_v": rank_v,
    "tucker_rank_head": tucker_rank_head,
    "tucker_rank_channel": tucker_rank_channel,
    "sinter_A": sinter_A,
    "sinter_omega": sinter_omega,
    "nonlinear_mlp_hidden_ratios": tested_mlp_ratios,
    "experiments": [e["name"] for e in experiments],
    "timestamp": datetime.datetime.now().isoformat(),
}

with open(os.path.join(run_dir, "hparams.json"), "w") as f:
    json.dump(hparams, f, indent=2)

# ============== 数据加载 ==============

print(f"\n\n######################## Experiments on {model_tag} ({model_name}) / {dataset_name} ########################")

train_loader, val_loader, num_classes = get_loaders(
    dataset_name=dataset_name,
    data_dir=data_dir,
    batch_size=batch_size,
    img_size=img_size,
    num_workers=num_workers,
)

# ============== 1) Baseline MHA 参数量 / 维度信息 ==============

print("\n==================== Baseline MHA model ====================")
baseline_model = ViTClassifier(
    num_classes=num_classes,
    model_name=model_name,
    pretrained=pretrained,
    attn_type="mha",
).to(device)

baseline_params = sum(p.numel() for p in baseline_model.parameters())
print(f"[{model_tag}] Baseline MHA params: {baseline_params / 1e6:.2f}M")

first_attn_baseline = baseline_model.vit.blocks[0].attn
if hasattr(first_attn_baseline, "dim"):
    dim = first_attn_baseline.dim
else:
    dim = first_attn_baseline.qkv.in_features

num_heads = first_attn_baseline.num_heads
head_dim = dim // num_heads
print(f"[{model_tag}] dim={dim}, num_heads={num_heads}, head_dim={head_dim}")

del baseline_model
if device == "cuda":
    torch.cuda.empty_cache()

# ============== 2) 依次跑多个实验 ==============

results_all = []      # 用来存最后总曲线图的信息

for exp in experiments:
    exp_name     = exp["name"]
    attn_type    = exp["attn_type"]
    total_epochs = exp["total_epochs"]
    mlp_ratio    = exp.get("mlp_ratio", None)

    print(f"\n==================== Experiment: {model_tag}-{exp_name} ====================")

    # 为当前实验建一个子目录
    exp_dir = os.path.join(run_dir, exp_name)
    os.makedirs(exp_dir, exist_ok=True)

    # ---------- 构建模型 ----------
    if attn_type == "mha":
        model = ViTClassifier(
            num_classes=num_classes,
            model_name=model_name,
            pretrained=pretrained,
            attn_type="mha",
        ).to(device)

    elif attn_type == "tpa":
        # True TPA（A/B 都 contextual）
        model = ViTClassifier(
            num_classes=num_classes,
            model_name=model_name,
            pretrained=pretrained,
            attn_type="tpa",
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
        ).to(device)

    elif attn_type == "nonlinear_tpa":
        # 非线性 TPA：Q = MLP(1/R * A^T B)
        model = ViTClassifier(
            num_classes=num_classes,
            model_name=model_name,
            pretrained=pretrained,
            attn_type="nonlinear_tpa",
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            nonlinear_mlp_hidden_ratio=mlp_ratio if mlp_ratio is not None else 1.0,
        ).to(device)

    elif attn_type == "sinter_tpa":
        # Sinter TPA：Q = MLP_Sinter(1/R * A^T B)
        model = ViTClassifier(
            num_classes=num_classes,
            model_name=model_name,
            pretrained=pretrained,
            attn_type="sinter_tpa",
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
            nonlinear_mlp_hidden_ratio=mlp_ratio if mlp_ratio is not None else 1.0,
            sinter_A=sinter_A,
            sinter_omega=sinter_omega,
        ).to(device)

    elif attn_type == "tucker_tpa":
        model = ViTClassifier(
            num_classes=num_classes,
            model_name=model_name,
            pretrained=pretrained,
            attn_type="tucker_tpa",
            rank_q=rank_q,
            rank_k=rank_k,
            rank_v=rank_v,
        ).to(device)

    else:
        raise ValueError(f"Unexpected attn_type: {attn_type}")

    total_params = sum(p.numel() for p in model.parameters())
    print(
        f"[{model_tag}-{exp_name}] Total params: {total_params / 1e6:.2f}M "
        f"(diff vs MHA: {(total_params - baseline_params)/1e6:.3f}M)"
    )

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    if use_amp and device == "cuda":
        scaler = torch.cuda.amp.GradScaler()
    else:
        scaler = None

    # ---------- 调用对应的训练函数 ----------
    if attn_type == "tucker_tpa":
        history = train_model_two_stage_tucker(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            total_epochs=total_epochs,
            ab_epochs=tucker_ab_epochs,
            scaler=scaler,
            is_tucker_model=True,
        )

    elif attn_type == "tpa":
        history = train_model_TPA(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            total_epochs=total_epochs,
            scaler=scaler,
        )

    elif attn_type == "nonlinear_tpa":
        history = train_model_nonlinear_TPA(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            total_epochs=total_epochs,
            scaler=scaler,
        )

    elif attn_type == "sinter_tpa":
        history = train_model_sinter_TPA(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            total_epochs=total_epochs,
            scaler=scaler,
        )

    elif attn_type == "mha":
        history = train_model_MHA(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            total_epochs=total_epochs,
            scaler=scaler,
        )

    else:
        raise ValueError(f"Unknown attn_type: {attn_type}")

    train_loss_curve = history["train_loss_curve"]
    val_loss_curve   = history["val_loss_curve"]
    train_acc_curve  = history["train_acc_curve"]
    val_acc_curve    = history["val_acc_curve"]
    best_val_acc     = history["best_val_acc"]

    print(f">>> [{model_tag}-{exp_name}] best val acc: {best_val_acc:.4f}")

    # ---------- KV cost 简易估算 ----------
    first_attn = model.vit.blocks[0].attn
    if hasattr(first_attn, "dim"):
        dim_attn = first_attn.dim
    else:
        dim_attn = first_attn.qkv.in_features
    num_heads_attn = first_attn.num_heads
    head_dim_attn = dim_attn // num_heads_attn

    kv_mha = 2 * num_heads_attn * head_dim_attn  # = 2 * dim

    if attn_type == "mha":
        normalized_kv_cost = 1.0
    elif attn_type in ("tpa", "nonlinear_tpa", "sinter_tpa"):
        kv_tpa = (rank_k + rank_v) * (num_heads_attn + head_dim_attn)
        normalized_kv_cost = kv_tpa / kv_mha
    elif attn_type == "tucker_tpa":
        normalized_kv_cost = 1.0
    else:
        normalized_kv_cost = 1.0

    # ---------- 把当前实验结果存起来 ----------
    results_all.append(
        {
            "name": exp_name,
            "attn_type": attn_type,
            "mlp_ratio": mlp_ratio,
            "params": total_params,
            "best_val_acc": best_val_acc,
            "train_loss_curve": train_loss_curve,
            "val_loss_curve": val_loss_curve,
            "train_acc_curve": train_acc_curve,
            "val_acc_curve": val_acc_curve,
            "kv_cost": normalized_kv_cost,
        }
    )

    # ---------- 写 log ----------
    log_lines = []
    # 第一行：模型参数量信息
    log_lines.append(
        f"Total params: {total_params} ({total_params/1e6:.2f}M), "
        f"diff vs baseline MHA: {total_params - baseline_params} "
        f"({(total_params - baseline_params)/1e6:.3f}M)"
    )
    if mlp_ratio is not None:
        log_lines.append(f"nonlinear_mlp_hidden_ratio: {mlp_ratio}")
    log_lines.append(f"attn_type: {attn_type}")
    log_lines.append(f"kv_cost (normalized to MHA=1): {normalized_kv_cost:.4f}")
    log_lines.append("")  # 空行分隔

    for ep in range(total_epochs):
        log_lines.append(
            f"Epoch {ep+1}/{total_epochs} | "
            f"train loss: {train_loss_curve[ep]:.4f}, train acc: {train_acc_curve[ep]:.3f} | "
            f"val loss: {val_loss_curve[ep]:.4f}, val acc: {val_acc_curve[ep]:.3f}"
        )

    exp_log_path = os.path.join(
        exp_dir,
        f"log_{dataset_name}_{model_tag}_{exp_name}_totEp{total_epochs}_lr{lr}_R{R_shared}_rq{rank_q}_rh{tucker_rank_head}_rd{tucker_rank_channel}.txt",
    )
    with open(exp_log_path, "w") as f:
        f.write("\n".join(log_lines))

    # ---------- 保存最终 checkpoint（最新一版） ----------
    ckpt_name = (
        f"ckpt_{dataset_name}_{model_tag}_{exp_name}"
        f"_totEp{total_epochs}_lr{lr}_R{R_shared}_rq{rank_q}_rh{tucker_rank_head}_rd{tucker_rank_channel}.pt"
    )
    ckpt_path = os.path.join(exp_dir, ckpt_name)

    torch.save(
        {
            "attn_type": attn_type,
            "total_epochs": total_epochs,
            "mlp_ratio": mlp_ratio,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "best_val_acc": best_val_acc,
            "train_loss_curve": train_loss_curve,
            "val_loss_curve": val_loss_curve,
            "train_acc_curve": train_acc_curve,
            "val_acc_curve": val_acc_curve,
            "hparams": hparams,
        },
        ckpt_path,
    )

    # ---------- 画当前实验的 loss 收敛图 ----------
    epochs_range = range(1, total_epochs + 1)

    plt.figure()
    plt.plot(epochs_range, train_loss_curve, label="train loss")
    plt.plot(epochs_range, val_loss_curve, label="val loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"{dataset_name} - {model_tag} - {exp_name}: loss convergence")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    loss_fig_path = os.path.join(
        exp_dir,
        f"loss_curve_{dataset_name}_{model_tag}_{exp_name}_totEp{total_epochs}.png",
    )
    plt.savefig(loss_fig_path)
    plt.show()

    # ---------- 当前实验的 val acc 曲线 ----------
    plt.figure()
    plt.plot(epochs_range, val_acc_curve, label=f"{exp_name} val acc")
    plt.xlabel("Epoch")
    plt.ylabel("Val accuracy")
    plt.title(f"{dataset_name} - {model_tag} - {exp_name}: val acc")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    acc_fig_path = os.path.join(
        exp_dir,
        f"val_acc_curve_{dataset_name}_{model_tag}_{exp_name}_totEp{total_epochs}.png",
    )
    plt.savefig(acc_fig_path)
    plt.show()

    # ---------- 截至当前 experiment 的所有实验对比图（val acc & val loss） ----------
    # Val accuracy comparison
    plt.figure()
    for res in results_all:
        total_epochs_exp = len(res["val_acc_curve"])
        epochs_range_all = range(1, total_epochs_exp + 1)
        label = res["name"]
        if res.get("mlp_ratio") is not None:
            label += f" (ratio={res['mlp_ratio']})"
        label += f" (KV={res['kv_cost']:.2f})"
        plt.plot(epochs_range_all, res["val_acc_curve"], label=label)
    plt.xlabel("Epoch")
    plt.ylabel("Val accuracy")
    plt.title(f"{dataset_name} - {model_tag}: Val acc comparison (up to {exp_name})")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    combined_acc_path = os.path.join(
        run_dir,
        f"combined_val_acc_upto_{exp_name}_BaseEp{epochs_base}_MHAEp{epochs_mha}_R{R_shared}.png",
    )
    plt.savefig(combined_acc_path)
    plt.show()

    # Val loss comparison
    plt.figure()
    for res in results_all:
        total_epochs_exp = len(res["val_loss_curve"])
        epochs_range_all = range(1, total_epochs_exp + 1)
        label = res["name"]
        if res.get("mlp_ratio") is not None:
            label += f" (ratio={res['mlp_ratio']})"
        label += f" (KV={res['kv_cost']:.2f})"
        plt.plot(epochs_range_all, res["val_loss_curve"], label=label)
    plt.xlabel("Epoch")
    plt.ylabel("Val loss")
    plt.title(f"{dataset_name} - {model_tag}: Val loss comparison (up to {exp_name})")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    combined_loss_path = os.path.join(
        run_dir,
        f"combined_val_loss_upto_{exp_name}_BaseEp{epochs_base}_MHAEp{epochs_mha}_R{R_shared}.png",
    )
    plt.savefig(combined_loss_path)
    plt.show()

    del model
    if device == "cuda":
        torch.cuda.empty_cache()

# ============== 3) 写 summary.json ==============

summary = []
for res in results_all:
    summary.append(
        {
            "name": res["name"],
            "attn_type": res["attn_type"],
            "mlp_ratio": res["mlp_ratio"],
            "params_million": res["params"] / 1e6,
            "best_val_acc": float(res["best_val_acc"]),
            "kv_cost": float(res["kv_cost"]),
            "epochs": len(res["val_acc_curve"]),
        }
    )

with open(os.path.join(run_dir, "summary.json"), "w") as f:
    json.dump(summary, f, indent=2)
